In [1]:
import numpy as np
import seaborn as sb
import pandas as pd
import matplotlib.pyplot as plt
from pandas import Grouper
from pandas import DataFrame
import csv
import pickle
import math
import matrixprofile as mp
from matplotlib.patches import Rectangle
from fbprophet import Prophet
from prophet.plot import plot_yearly
from sklearn.metrics import mean_squared_error
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics
from sklearn.model_selection import TimeSeriesSplit
from numpy import sqrt
import datetime
from datetime import timedelta
import plotly.graph_objs as go
from sklearn.metrics import r2_score
import re

/home/jonathan/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# 1) Datos Metereologicos Mayo

In [2]:
DataMeteo = pd.read_csv("/home/jonathan/tesis/Data_Temperatura_Lluvia/may_meteo20.csv", sep=';')
DataMeteo.head(1)

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H01,V01,...,H20,V20,H21,V21,H22,V22,H23,V23,H24,V24
0,28,79,102,81,28079102_81_98,2020,5,1,2.3,V,...,3.82,V,2.58,V,1.8,V,1.17,V,1.47,V


# 1.1) Procesamiento Datos de Temperatura
## Obtenemos datos de temperatura de la estación Chamberí
#### Punto de muestreo de Chamartín 28079111, estación 111 
#### Punto de muestreo de Centro 28079110, estación 110
#### Punto de muestreo de Chamberí 28079109, estación 109
#### Debido a que no se encuentra la estación 111 se procede a utilizar los datos de la estación Chamberí

In [3]:
DataFiltrar = DataMeteo.loc[DataMeteo.loc[:,'ESTACION'] == 109]
DataTemp = DataFiltrar.loc[DataFiltrar.loc[:,'MAGNITUD'] == 83]
DataTemp = DataTemp.drop(columns=['PROVINCIA','MUNICIPIO','ESTACION','MAGNITUD','V01','V02','V03','V04','V05','V06','V07','V08','V09','V10','V11','V12','V13','V14','V15','V16','V17','V18','V19','V20','V21','V22','V23','V24'])
TableTemp = DataTemp
# Convertimos los elementos de las columnas "ANO" "MES" "DIA" a string
TableTemp['ANO']= TableTemp['ANO'].map(str)
TableTemp['MES']= TableTemp['MES'].map(str)
TableTemp['DIA']= TableTemp['DIA'].map(str)
# Ajustamos la columna ds con (-) para posteriormente obtener un timestamp
TableTemp['ds'] = TableTemp['ANO'] + '-' + TableTemp['MES'] + '-' + TableTemp['DIA']
TableTemp.tail(5)

,PUNTO_MUESTREO,ANO,MES,DIA,H01,H02,H03,H04,H05,H06,...,H16,H17,H18,H19,H20,H21,H22,H23,H24,ds
1173,28079109_83_98,2020,5,27,21.2,20.5,20.1,19.8,19.4,18.9,...,29.0,29.6,30.2,29.1,28.0,26.4,24.9,23.7,22.5,2020-5-27
1174,28079109_83_98,2020,5,28,21.5,20.9,19.9,19.2,18.8,18.6,...,29.0,28.7,28.9,28.4,27.4,26.5,25.3,24.3,23.2,2020-5-28
1175,28079109_83_98,2020,5,29,22.3,21.2,20.4,19.6,19.1,18.9,...,31.3,30.9,31.9,30.7,29.4,28.5,27.8,27.1,26.2,2020-5-29
1176,28079109_83_98,2020,5,30,25.1,24.3,22.9,21.6,21.0,20.2,...,31.9,32.1,33.1,31.4,30.4,29.6,27.8,26.1,25.7,2020-5-30
1177,28079109_83_98,2020,5,31,25.0,24.5,23.6,22.5,21.7,21.2,...,31.5,32.2,26.3,25.7,18.3,19.7,19.4,19.4,18.9,2020-5-31


# Obtenemos tablas de las 24 horas TableTemp#

In [4]:
######### TableTemp1 ##############
TableTemp1 = pd.pivot_table(TableTemp, values=['H01'], index=["ds"], aggfunc={'H01':'first'})
TableTemp1 = TableTemp1.reset_index()
TableTemp1['ds'] = TableTemp1['ds'] + '-' + '010000'
TableTemp1['ds'] = pd.to_datetime(TableTemp1['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp1 = TableTemp1.set_index('ds')
TableTemp1 = TableTemp1.sort_index()
TableTemp1 = TableTemp1.rename(columns={'H01':'Temp'})
######### TableTemp2 ##############
TableTemp2 = pd.pivot_table(TableTemp, values=['H02'], index=["ds"], aggfunc={'H02':'first'})
TableTemp2 = TableTemp2.reset_index()
TableTemp2['ds'] = TableTemp2['ds'] + '-' + '020000'
TableTemp2['ds'] = pd.to_datetime(TableTemp2['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp2 = TableTemp2.set_index('ds')
TableTemp2 = TableTemp2.sort_index()
TableTemp2 = TableTemp2.rename(columns={'H02':'Temp'})
######### TableTemp3 ##############
TableTemp3 = pd.pivot_table(TableTemp, values=['H03'], index=["ds"], aggfunc={'H03':'first'})
TableTemp3 = TableTemp3.reset_index()
TableTemp3['ds'] = TableTemp3['ds'] + '-' + '030000'
TableTemp3['ds'] = pd.to_datetime(TableTemp3['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp3 = TableTemp3.set_index('ds')
TableTemp3 = TableTemp3.sort_index()
TableTemp3 = TableTemp3.rename(columns={'H03':'Temp'})
######### TableTemp4 ##############
TableTemp4 = pd.pivot_table(TableTemp, values=['H04'], index=["ds"], aggfunc={'H04':'first'})
TableTemp4 = TableTemp4.reset_index()
TableTemp4['ds'] = TableTemp4['ds'] + '-' + '040000'
TableTemp4['ds'] = pd.to_datetime(TableTemp4['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp4 = TableTemp4.set_index('ds')
TableTemp4 = TableTemp4.sort_index()
TableTemp4 = TableTemp4.rename(columns={'H04':'Temp'})
######### TableTemp5 ##############
TableTemp5 = pd.pivot_table(TableTemp, values=['H05'], index=["ds"], aggfunc={'H05':'first'})
TableTemp5 = TableTemp5.reset_index()
TableTemp5['ds'] = TableTemp5['ds'] + '-' + '050000'
TableTemp5['ds'] = pd.to_datetime(TableTemp5['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp5 = TableTemp5.set_index('ds')
TableTemp5 = TableTemp5.sort_index()
TableTemp5 = TableTemp5.rename(columns={'H05':'Temp'})
######### TableTemp6 ##############
TableTemp6 = pd.pivot_table(TableTemp, values=['H06'], index=["ds"], aggfunc={'H06':'first'})
TableTemp6 = TableTemp6.reset_index()
TableTemp6['ds'] = TableTemp6['ds'] + '-' + '060000'
TableTemp6['ds'] = pd.to_datetime(TableTemp6['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp6 = TableTemp6.set_index('ds')
TableTemp6 = TableTemp6.sort_index()
TableTemp6 = TableTemp6.rename(columns={'H06':'Temp'})
######### TableTemp7 ##############
TableTemp7 = pd.pivot_table(TableTemp, values=['H07'], index=["ds"], aggfunc={'H07':'first'})
TableTemp7 = TableTemp7.reset_index()
TableTemp7['ds'] = TableTemp7['ds'] + '-' + '070000'
TableTemp7['ds'] = pd.to_datetime(TableTemp7['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp7 = TableTemp7.set_index('ds')
TableTemp7 = TableTemp7.sort_index()
TableTemp7 = TableTemp7.rename(columns={'H07':'Temp'})
######### TableTemp8 ##############
TableTemp8 = pd.pivot_table(TableTemp, values=['H08'], index=["ds"], aggfunc={'H08':'first'})
TableTemp8 = TableTemp8.reset_index()
TableTemp8['ds'] = TableTemp8['ds'] + '-' + '080000'
TableTemp8['ds'] = pd.to_datetime(TableTemp8['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp8 = TableTemp8.set_index('ds')
TableTemp8 = TableTemp8.sort_index()
TableTemp8 = TableTemp8.rename(columns={'H08':'Temp'})
######### TableTemp9 ##############
TableTemp9 = pd.pivot_table(TableTemp, values=['H09'], index=["ds"], aggfunc={'H09':'first'})
TableTemp9 = TableTemp9.reset_index()
TableTemp9['ds'] = TableTemp9['ds'] + '-' + '090000'
TableTemp9['ds'] = pd.to_datetime(TableTemp9['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp9 = TableTemp9.set_index('ds')
TableTemp9 = TableTemp9.sort_index()
TableTemp9 = TableTemp9.rename(columns={'H09':'Temp'})
######### TableTemp10 ##############
TableTemp10 = pd.pivot_table(TableTemp, values=['H10'], index=["ds"], aggfunc={'H10':'first'})
TableTemp10 = TableTemp10.reset_index()
TableTemp10['ds'] = TableTemp10['ds'] + '-' + '100000'
TableTemp10['ds'] = pd.to_datetime(TableTemp10['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp10 = TableTemp10.set_index('ds')
TableTemp10 = TableTemp10.sort_index()
TableTemp10 = TableTemp10.rename(columns={'H10':'Temp'})
######### TableTemp11 ##############
TableTemp11 = pd.pivot_table(TableTemp, values=['H11'], index=["ds"], aggfunc={'H11':'first'})
TableTemp11 = TableTemp11.reset_index()
TableTemp11['ds'] = TableTemp11['ds'] + '-' + '110000'
TableTemp11['ds'] = pd.to_datetime(TableTemp11['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp11 = TableTemp11.set_index('ds')
TableTemp11 = TableTemp11.sort_index()
TableTemp11 = TableTemp11.rename(columns={'H11':'Temp'})
######### TableTemp12 ##############
TableTemp12 = pd.pivot_table(TableTemp, values=['H12'], index=["ds"], aggfunc={'H12':'first'})
TableTemp12 = TableTemp12.reset_index()
TableTemp12['ds'] = TableTemp12['ds'] + '-' + '120000'
TableTemp12['ds'] = pd.to_datetime(TableTemp12['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp12 = TableTemp12.set_index('ds')
TableTemp12 = TableTemp12.sort_index()
TableTemp12 = TableTemp12.rename(columns={'H12':'Temp'})
######### TableTemp13 ##############
TableTemp13 = pd.pivot_table(TableTemp, values=['H13'], index=["ds"], aggfunc={'H13':'first'})
TableTemp13 = TableTemp13.reset_index()
TableTemp13['ds'] = TableTemp13['ds'] + '-' + '130000'
TableTemp13['ds'] = pd.to_datetime(TableTemp13['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp13 = TableTemp13.set_index('ds')
TableTemp13 = TableTemp13.sort_index()
TableTemp13 = TableTemp13.rename(columns={'H13':'Temp'})
######### TableTemp14 ##############
TableTemp14 = pd.pivot_table(TableTemp, values=['H14'], index=["ds"], aggfunc={'H14':'first'})
TableTemp14 = TableTemp14.reset_index()
TableTemp14['ds'] = TableTemp14['ds'] + '-' + '140000'
TableTemp14['ds'] = pd.to_datetime(TableTemp14['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp14 = TableTemp14.set_index('ds')
TableTemp14 = TableTemp14.sort_index()
TableTemp14 = TableTemp14.rename(columns={'H14':'Temp'})
######### TableTemp15 ##############
TableTemp15 = pd.pivot_table(TableTemp, values=['H15'], index=["ds"], aggfunc={'H15':'first'})
TableTemp15 = TableTemp15.reset_index()
TableTemp15['ds'] = TableTemp15['ds'] + '-' + '150000'
TableTemp15['ds'] = pd.to_datetime(TableTemp15['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp15 = TableTemp15.set_index('ds')
TableTemp15 = TableTemp15.sort_index()
TableTemp15 = TableTemp15.rename(columns={'H15':'Temp'})
######### TableTemp16 ##############
TableTemp16 = pd.pivot_table(TableTemp, values=['H16'], index=["ds"], aggfunc={'H16':'first'})
TableTemp16 = TableTemp16.reset_index()
TableTemp16['ds'] = TableTemp16['ds'] + '-' + '160000'
TableTemp16['ds'] = pd.to_datetime(TableTemp16['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp16 = TableTemp16.set_index('ds')
TableTemp16 = TableTemp16.sort_index()
TableTemp16 = TableTemp16.rename(columns={'H16':'Temp'})
######### TableTemp17 ##############
TableTemp17 = pd.pivot_table(TableTemp, values=['H17'], index=["ds"], aggfunc={'H17':'first'})
TableTemp17 = TableTemp17.reset_index()
TableTemp17['ds'] = TableTemp17['ds'] + '-' + '170000'
TableTemp17['ds'] = pd.to_datetime(TableTemp17['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp17 = TableTemp17.set_index('ds')
TableTemp17 = TableTemp17.sort_index()
TableTemp17 = TableTemp17.rename(columns={'H17':'Temp'})
######### TableTemp18 ##############
TableTemp18 = pd.pivot_table(TableTemp, values=['H18'], index=["ds"], aggfunc={'H18':'first'})
TableTemp18 = TableTemp18.reset_index()
TableTemp18['ds'] = TableTemp18['ds'] + '-' + '180000'
TableTemp18['ds'] = pd.to_datetime(TableTemp18['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp18 = TableTemp18.set_index('ds')
TableTemp18 = TableTemp18.sort_index()
TableTemp18 = TableTemp18.rename(columns={'H18':'Temp'})
######### TableTemp19 ##############
TableTemp19 = pd.pivot_table(TableTemp, values=['H19'], index=["ds"], aggfunc={'H19':'first'})
TableTemp19 = TableTemp19.reset_index()
TableTemp19['ds'] = TableTemp19['ds'] + '-' + '190000'
TableTemp19['ds'] = pd.to_datetime(TableTemp19['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp19 = TableTemp19.set_index('ds')
TableTemp19 = TableTemp19.sort_index()
TableTemp19 = TableTemp19.rename(columns={'H19':'Temp'})
######### TableTemp20 ##############
TableTemp20 = pd.pivot_table(TableTemp, values=['H20'], index=["ds"], aggfunc={'H20':'first'})
TableTemp20 = TableTemp20.reset_index()
TableTemp20['ds'] = TableTemp20['ds'] + '-' + '200000'
TableTemp20['ds'] = pd.to_datetime(TableTemp20['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp20 = TableTemp20.set_index('ds')
TableTemp20 = TableTemp20.sort_index()
TableTemp20 = TableTemp20.rename(columns={'H20':'Temp'})
######### TableTemp21 ##############
TableTemp21 = pd.pivot_table(TableTemp, values=['H21'], index=["ds"], aggfunc={'H21':'first'})
TableTemp21 = TableTemp21.reset_index()
TableTemp21['ds'] = TableTemp21['ds'] + '-' + '210000'
TableTemp21['ds'] = pd.to_datetime(TableTemp21['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp21 = TableTemp21.set_index('ds')
TableTemp21 = TableTemp21.sort_index()
TableTemp21 = TableTemp21.rename(columns={'H21':'Temp'})
######### TableTemp22 ##############
TableTemp22 = pd.pivot_table(TableTemp, values=['H22'], index=["ds"], aggfunc={'H22':'first'})
TableTemp22 = TableTemp22.reset_index()
TableTemp22['ds'] = TableTemp22['ds'] + '-' + '220000'
TableTemp22['ds'] = pd.to_datetime(TableTemp22['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp22 = TableTemp22.set_index('ds')
TableTemp22 = TableTemp22.sort_index()
TableTemp22 = TableTemp22.rename(columns={'H22':'Temp'})
######### TableTemp23 ##############
TableTemp23 = pd.pivot_table(TableTemp, values=['H23'], index=["ds"], aggfunc={'H23':'first'})
TableTemp23 = TableTemp23.reset_index()
TableTemp23['ds'] = TableTemp23['ds'] + '-' + '230000'
TableTemp23['ds'] = pd.to_datetime(TableTemp23['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp23 = TableTemp23.set_index('ds')
TableTemp23 = TableTemp23.sort_index()
TableTemp23 = TableTemp23.rename(columns={'H23':'Temp'})
######### TableTemp0 ##############
TableTemp0 = pd.pivot_table(TableTemp, values=['H24'], index=["ds"], aggfunc={'H24':'first'})
TableTemp0 = TableTemp0.reset_index()
TableTemp0['ds'] = TableTemp0['ds'] + '-' + '000000'
TableTemp0['ds'] = pd.to_datetime(TableTemp0['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp0 = TableTemp0.set_index('ds')
TableTemp0 = TableTemp0.sort_index()
TableTemp0 = TableTemp0.rename(columns={'H24':'Temp'})

# Concatenamos todas las horas en una sola tabla

In [5]:
Data_TempMay = pd.concat([TableTemp0,TableTemp1,TableTemp2,TableTemp3,TableTemp4,TableTemp5,TableTemp6,TableTemp7,TableTemp8,TableTemp9,TableTemp10,TableTemp11,TableTemp12,TableTemp13,TableTemp14,TableTemp15,TableTemp16,TableTemp17,TableTemp18,TableTemp19,TableTemp20,TableTemp21,TableTemp22,TableTemp23])
Data_TempMay = Data_TempMay.sort_index()
Data_TempMay.tail(5)

,Temp
ds,
2020-05-31 19:00:00,25.7
2020-05-31 20:00:00,18.3
2020-05-31 21:00:00,19.7
2020-05-31 22:00:00,19.4
2020-05-31 23:00:00,19.4


# 1.2) Procesamiento Datos Precipitación
## Obtenemos datos de Precipitación de la estación Hortaleza 
## Las estaciones que tienen datos de precipitación son las siguientes: 102,103,106,107,108,24,39,54,56,59

In [6]:
DataFiltrar2 = DataMeteo.loc[DataMeteo.loc[:,'ESTACION'] == 107]
DataPrec = DataFiltrar2.loc[DataFiltrar2.loc[:,'MAGNITUD'] == 89]
DataPrec = DataPrec.drop(columns=['PROVINCIA','MUNICIPIO','ESTACION','MAGNITUD','V01','V02','V03','V04','V05','V06','V07','V08','V09','V10','V11','V12','V13','V14','V15','V16','V17','V18','V19','V20','V21','V22','V23','V24'])
TablePrec = DataPrec
# Convertimos los elementos de las columnas "ANO" "MES" "DIA" a string
TablePrec['ANO']= TablePrec['ANO'].map(str)
TablePrec['MES']= TablePrec['MES'].map(str)
TablePrec['DIA']= TablePrec['DIA'].map(str)
# Ajustamos la columna ds con (-) para posteriormente obtener un timestamp
TablePrec['ds'] = TablePrec['ANO'] + '-' + TablePrec['MES'] + '-' + TablePrec['DIA']
TablePrec.tail(5)

,PUNTO_MUESTREO,ANO,MES,DIA,H01,H02,H03,H04,H05,H06,...,H16,H17,H18,H19,H20,H21,H22,H23,H24,ds
925,28079107_89_98,2020,5,27,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-5-27
926,28079107_89_98,2020,5,28,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-5-28
927,28079107_89_98,2020,5,29,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-5-29
928,28079107_89_98,2020,5,30,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-5-30
929,28079107_89_98,2020,5,31,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.1,0.2,13.5,0.2,0.1,0.0,0.0,2020-5-31


# Obtenemos tablas de las 24 horas TablePrec#

In [7]:
######### TableTemp1 ##############
TablePrec1 = pd.pivot_table(TablePrec, values=['H01'], index=["ds"], aggfunc={'H01':'first'})
TablePrec1 = TablePrec1.reset_index()
TablePrec1['ds'] = TablePrec1['ds'] + '-' + '010000'
TablePrec1['ds'] = pd.to_datetime(TablePrec1['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec1 = TablePrec1.set_index('ds')
TablePrec1 = TablePrec1.sort_index()
TablePrec1 = TablePrec1.rename(columns={'H01':'Prec'})
######### TableTemp2 ##############
TablePrec2 = pd.pivot_table(TablePrec, values=['H02'], index=["ds"], aggfunc={'H02':'first'})
TablePrec2 = TablePrec2.reset_index()
TablePrec2['ds'] = TablePrec2['ds'] + '-' + '020000'
TablePrec2['ds'] = pd.to_datetime(TablePrec2['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec2 = TablePrec2.set_index('ds')
TablePrec2 = TablePrec2.sort_index()
TablePrec2 = TablePrec2.rename(columns={'H02':'Prec'})
######### TableTemp3 ##############
TablePrec3 = pd.pivot_table(TablePrec, values=['H03'], index=["ds"], aggfunc={'H03':'first'})
TablePrec3 = TablePrec3.reset_index()
TablePrec3['ds'] = TablePrec3['ds'] + '-' + '030000'
TablePrec3['ds'] = pd.to_datetime(TablePrec3['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec3 = TablePrec3.set_index('ds')
TablePrec3 = TablePrec3.sort_index()
TablePrec3 = TablePrec3.rename(columns={'H03':'Prec'})
######### TableTemp4 ##############
TablePrec4 = pd.pivot_table(TablePrec, values=['H04'], index=["ds"], aggfunc={'H04':'first'})
TablePrec4 = TablePrec4.reset_index()
TablePrec4['ds'] = TablePrec4['ds'] + '-' + '040000'
TablePrec4['ds'] = pd.to_datetime(TablePrec4['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec4 = TablePrec4.set_index('ds')
TablePrec4 = TablePrec4.sort_index()
TablePrec4 = TablePrec4.rename(columns={'H04':'Prec'})
######### TableTemp5 ##############
TablePrec5 = pd.pivot_table(TablePrec, values=['H05'], index=["ds"], aggfunc={'H05':'first'})
TablePrec5 = TablePrec5.reset_index()
TablePrec5['ds'] = TablePrec5['ds'] + '-' + '050000'
TablePrec5['ds'] = pd.to_datetime(TablePrec5['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec5 = TablePrec5.set_index('ds')
TablePrec5 = TablePrec5.sort_index()
TablePrec5 = TablePrec5.rename(columns={'H05':'Prec'})
######### TableTemp6 ##############
TablePrec6 = pd.pivot_table(TablePrec, values=['H06'], index=["ds"], aggfunc={'H06':'first'})
TablePrec6 = TablePrec6.reset_index()
TablePrec6['ds'] = TablePrec6['ds'] + '-' + '060000'
TablePrec6['ds'] = pd.to_datetime(TablePrec6['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec6 = TablePrec6.set_index('ds')
TablePrec6 = TablePrec6.sort_index()
TablePrec6 = TablePrec6.rename(columns={'H06':'Prec'})
######### TableTemp7 ##############
TablePrec7 = pd.pivot_table(TablePrec, values=['H07'], index=["ds"], aggfunc={'H07':'first'})
TablePrec7 = TablePrec7.reset_index()
TablePrec7['ds'] = TablePrec7['ds'] + '-' + '070000'
TablePrec7['ds'] = pd.to_datetime(TablePrec7['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec7 = TablePrec7.set_index('ds')
TablePrec7 = TablePrec7.sort_index()
TablePrec7 = TablePrec7.rename(columns={'H07':'Prec'})
######### TableTemp8 ##############
TablePrec8 = pd.pivot_table(TablePrec, values=['H08'], index=["ds"], aggfunc={'H08':'first'})
TablePrec8 = TablePrec8.reset_index()
TablePrec8['ds'] = TablePrec8['ds'] + '-' + '080000'
TablePrec8['ds'] = pd.to_datetime(TablePrec8['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec8 = TablePrec8.set_index('ds')
TablePrec8 = TablePrec8.sort_index()
TablePrec8 = TablePrec8.rename(columns={'H08':'Prec'})
######### TableTemp9 ##############
TablePrec9 = pd.pivot_table(TablePrec, values=['H09'], index=["ds"], aggfunc={'H09':'first'})
TablePrec9 = TablePrec9.reset_index()
TablePrec9['ds'] = TablePrec9['ds'] + '-' + '090000'
TablePrec9['ds'] = pd.to_datetime(TablePrec9['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec9 = TablePrec9.set_index('ds')
TablePrec9 = TablePrec9.sort_index()
TablePrec9 = TablePrec9.rename(columns={'H09':'Prec'})
######### TableTemp10 ##############
TablePrec10 = pd.pivot_table(TablePrec, values=['H10'], index=["ds"], aggfunc={'H10':'first'})
TablePrec10 = TablePrec10.reset_index()
TablePrec10['ds'] = TablePrec10['ds'] + '-' + '100000'
TablePrec10['ds'] = pd.to_datetime(TablePrec10['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec10 = TablePrec10.set_index('ds')
TablePrec10 = TablePrec10.sort_index()
TablePrec10 = TablePrec10.rename(columns={'H10':'Prec'})
######### TableTemp11 ##############
TablePrec11 = pd.pivot_table(TablePrec, values=['H11'], index=["ds"], aggfunc={'H11':'first'})
TablePrec11 = TablePrec11.reset_index()
TablePrec11['ds'] = TablePrec11['ds'] + '-' + '110000'
TablePrec11['ds'] = pd.to_datetime(TablePrec11['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec11 = TablePrec11.set_index('ds')
TablePrec11 = TablePrec11.sort_index()
TablePrec11 = TablePrec11.rename(columns={'H11':'Prec'})
######### TableTemp12 ##############
TablePrec12 = pd.pivot_table(TablePrec, values=['H12'], index=["ds"], aggfunc={'H12':'first'})
TablePrec12 = TablePrec12.reset_index()
TablePrec12['ds'] = TablePrec12['ds'] + '-' + '120000'
TablePrec12['ds'] = pd.to_datetime(TablePrec12['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec12 = TablePrec12.set_index('ds')
TablePrec12 = TablePrec12.sort_index()
TablePrec12 = TablePrec12.rename(columns={'H12':'Prec'})
######### TableTemp13 ##############
TablePrec13 = pd.pivot_table(TablePrec, values=['H13'], index=["ds"], aggfunc={'H13':'first'})
TablePrec13 = TablePrec13.reset_index()
TablePrec13['ds'] = TablePrec13['ds'] + '-' + '130000'
TablePrec13['ds'] = pd.to_datetime(TablePrec13['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec13 = TablePrec13.set_index('ds')
TablePrec13 = TablePrec13.sort_index()
TablePrec13 = TablePrec13.rename(columns={'H13':'Prec'})
######### TableTemp14 ##############
TablePrec14 = pd.pivot_table(TablePrec, values=['H14'], index=["ds"], aggfunc={'H14':'first'})
TablePrec14 = TablePrec14.reset_index()
TablePrec14['ds'] = TablePrec14['ds'] + '-' + '140000'
TablePrec14['ds'] = pd.to_datetime(TablePrec14['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec14 = TablePrec14.set_index('ds')
TablePrec14 = TablePrec14.sort_index()
TablePrec14 = TablePrec14.rename(columns={'H14':'Prec'})
######### TableTemp15 ##############
TablePrec15 = pd.pivot_table(TablePrec, values=['H15'], index=["ds"], aggfunc={'H15':'first'})
TablePrec15 = TablePrec15.reset_index()
TablePrec15['ds'] = TablePrec15['ds'] + '-' + '150000'
TablePrec15['ds'] = pd.to_datetime(TablePrec15['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec15 = TablePrec15.set_index('ds')
TablePrec15 = TablePrec15.sort_index()
TablePrec15 = TablePrec15.rename(columns={'H15':'Prec'})
######### TableTemp16 ##############
TablePrec16 = pd.pivot_table(TablePrec, values=['H16'], index=["ds"], aggfunc={'H16':'first'})
TablePrec16 = TablePrec16.reset_index()
TablePrec16['ds'] = TablePrec16['ds'] + '-' + '160000'
TablePrec16['ds'] = pd.to_datetime(TablePrec16['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec16 = TablePrec16.set_index('ds')
TablePrec16 = TablePrec16.sort_index()
TablePrec16 = TablePrec16.rename(columns={'H16':'Prec'})
######### TableTemp17 ##############
TablePrec17 = pd.pivot_table(TablePrec, values=['H17'], index=["ds"], aggfunc={'H17':'first'})
TablePrec17 = TablePrec17.reset_index()
TablePrec17['ds'] = TablePrec17['ds'] + '-' + '170000'
TablePrec17['ds'] = pd.to_datetime(TablePrec17['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec17 = TablePrec17.set_index('ds')
TablePrec17 = TablePrec17.sort_index()
TablePrec17 = TablePrec17.rename(columns={'H17':'Prec'})
######### TableTemp18 ##############
TablePrec18 = pd.pivot_table(TablePrec, values=['H18'], index=["ds"], aggfunc={'H18':'first'})
TablePrec18 = TablePrec18.reset_index()
TablePrec18['ds'] = TablePrec18['ds'] + '-' + '180000'
TablePrec18['ds'] = pd.to_datetime(TablePrec18['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec18 = TablePrec18.set_index('ds')
TablePrec18 = TablePrec18.sort_index()
TablePrec18 = TablePrec18.rename(columns={'H18':'Prec'})
######### TableTemp19 ##############
TablePrec19 = pd.pivot_table(TablePrec, values=['H19'], index=["ds"], aggfunc={'H19':'first'})
TablePrec19 = TablePrec19.reset_index()
TablePrec19['ds'] = TablePrec19['ds'] + '-' + '190000'
TablePrec19['ds'] = pd.to_datetime(TablePrec19['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec19 = TablePrec19.set_index('ds')
TablePrec19 = TablePrec19.sort_index()
TablePrec19 = TablePrec19.rename(columns={'H19':'Prec'})
######### TableTemp20 ##############
TablePrec20 = pd.pivot_table(TablePrec, values=['H20'], index=["ds"], aggfunc={'H20':'first'})
TablePrec20 = TablePrec20.reset_index()
TablePrec20['ds'] = TablePrec20['ds'] + '-' + '200000'
TablePrec20['ds'] = pd.to_datetime(TablePrec20['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec20 = TablePrec20.set_index('ds')
TablePrec20 = TablePrec20.sort_index()
TablePrec20 = TablePrec20.rename(columns={'H20':'Prec'})
######### TableTemp21 ##############
TablePrec21 = pd.pivot_table(TablePrec, values=['H21'], index=["ds"], aggfunc={'H21':'first'})
TablePrec21 = TablePrec21.reset_index()
TablePrec21['ds'] = TablePrec21['ds'] + '-' + '210000'
TablePrec21['ds'] = pd.to_datetime(TablePrec21['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec21 = TablePrec21.set_index('ds')
TablePrec21 = TablePrec21.sort_index()
TablePrec21 = TablePrec21.rename(columns={'H21':'Prec'})
######### TableTemp22 ##############
TablePrec22 = pd.pivot_table(TablePrec, values=['H22'], index=["ds"], aggfunc={'H22':'first'})
TablePrec22 = TablePrec22.reset_index()
TablePrec22['ds'] = TablePrec22['ds'] + '-' + '220000'
TablePrec22['ds'] = pd.to_datetime(TablePrec22['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec22 = TablePrec22.set_index('ds')
TablePrec22 = TablePrec22.sort_index()
TablePrec22 = TablePrec22.rename(columns={'H22':'Prec'})
######### TableTemp23 ##############
TablePrec23 = pd.pivot_table(TablePrec, values=['H23'], index=["ds"], aggfunc={'H23':'first'})
TablePrec23 = TablePrec23.reset_index()
TablePrec23['ds'] = TablePrec23['ds'] + '-' + '230000'
TablePrec23['ds'] = pd.to_datetime(TablePrec23['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec23 = TablePrec23.set_index('ds')
TablePrec23 = TablePrec23.sort_index()
TablePrec23 = TablePrec23.rename(columns={'H23':'Prec'})
######### TableTemp0 ##############
TablePrec0 = pd.pivot_table(TablePrec, values=['H24'], index=["ds"], aggfunc={'H24':'first'})
TablePrec0 = TablePrec0.reset_index()
TablePrec0['ds'] = TablePrec0['ds'] + '-' + '000000'
TablePrec0['ds'] = pd.to_datetime(TablePrec0['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec0 = TablePrec0.set_index('ds')
TablePrec0 = TablePrec0.sort_index()
TablePrec0 = TablePrec0.rename(columns={'H24':'Prec'})

In [8]:
Data_PrecMay = pd.concat([TablePrec0,TablePrec1,TablePrec2,TablePrec3,TablePrec4,TablePrec5,TablePrec6,TablePrec7,TablePrec8,TablePrec9,TablePrec10,TablePrec11,TablePrec12,TablePrec13,TablePrec14,TablePrec15,TablePrec16,TablePrec17,TablePrec18,TablePrec19,TablePrec20,TablePrec21,TablePrec22,TablePrec23])
Data_PrecMay = Data_PrecMay.sort_index()
Data_PrecMay.tail(5)

,Prec
ds,
2020-05-31 19:00:00,0.2
2020-05-31 20:00:00,13.5
2020-05-31 21:00:00,0.2
2020-05-31 22:00:00,0.1
2020-05-31 23:00:00,0.0


# 2) Datos Metereologicos Junio

In [9]:
DataMeteo = pd.read_csv("/home/jonathan/tesis/Data_Temperatura_Lluvia/jun_meteo20.csv", sep=';')
DataMeteo.head(1)

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H01,V01,...,H20,V20,H21,V21,H22,V22,H23,V23,H24,V24
0,28,79,102,81,28079102_81_98,2020,6,1,3.2,V,...,1.2,V,1.63,V,1.55,V,1.33,V,1.2,V


# 2.1) Procesamiento Datos de Temperatura
## Obtenemos datos de temperatura de la estación Chamberí

In [10]:
DataFiltrar = DataMeteo.loc[DataMeteo.loc[:,'ESTACION'] == 109]
DataTemp = DataFiltrar.loc[DataFiltrar.loc[:,'MAGNITUD'] == 83]
DataTemp = DataTemp.drop(columns=['PROVINCIA','MUNICIPIO','ESTACION','MAGNITUD','V01','V02','V03','V04','V05','V06','V07','V08','V09','V10','V11','V12','V13','V14','V15','V16','V17','V18','V19','V20','V21','V22','V23','V24'])
TableTemp = DataTemp
# Convertimos los elementos de las columnas "ANO" "MES" "DIA" a string
TableTemp['ANO']= TableTemp['ANO'].map(str)
TableTemp['MES']= TableTemp['MES'].map(str)
TableTemp['DIA']= TableTemp['DIA'].map(str)
# Ajustamos la columna ds con (-) para posteriormente obtener un timestamp
TableTemp['ds'] = TableTemp['ANO'] + '-' + TableTemp['MES'] + '-' + TableTemp['DIA']
TableTemp.tail(5)

,PUNTO_MUESTREO,ANO,MES,DIA,H01,H02,H03,H04,H05,H06,...,H16,H17,H18,H19,H20,H21,H22,H23,H24,ds
1009,28079109_83_98,2020,6,26,25.3,24.6,24.0,23.7,23.3,22.9,...,30.8,31.1,31.1,30.3,29.0,28.0,26.8,25.7,24.8,2020-6-26
1010,28079109_83_98,2020,6,27,24.0,23.3,22.8,22.2,21.5,21.1,...,31.5,32.5,32.3,31.4,31.0,30.2,29.0,27.7,26.7,2020-6-27
1011,28079109_83_98,2020,6,28,26.1,25.3,24.9,24.4,23.8,23.4,...,33.8,34.6,34.5,34.2,32.9,32.2,31.4,30.1,29.1,2020-6-28
1012,28079109_83_98,2020,6,29,28.3,27.4,26.4,25.7,25.7,25.2,...,35.6,36.5,36.7,36.0,34.8,33.9,32.8,31.4,30.3,2020-6-29
1013,28079109_83_98,2020,6,30,29.3,28.2,26.6,25.6,24.9,24.3,...,36.8,37.0,37.2,36.6,35.4,34.4,33.0,31.5,30.7,2020-6-30


# Obtenemos tablas de las 24 horas TableTemp#

In [11]:
######### TableTemp1 ##############
TableTemp1 = pd.pivot_table(TableTemp, values=['H01'], index=["ds"], aggfunc={'H01':'first'})
TableTemp1 = TableTemp1.reset_index()
TableTemp1['ds'] = TableTemp1['ds'] + '-' + '010000'
TableTemp1['ds'] = pd.to_datetime(TableTemp1['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp1 = TableTemp1.set_index('ds')
TableTemp1 = TableTemp1.sort_index()
TableTemp1 = TableTemp1.rename(columns={'H01':'Temp'})
######### TableTemp2 ##############
TableTemp2 = pd.pivot_table(TableTemp, values=['H02'], index=["ds"], aggfunc={'H02':'first'})
TableTemp2 = TableTemp2.reset_index()
TableTemp2['ds'] = TableTemp2['ds'] + '-' + '020000'
TableTemp2['ds'] = pd.to_datetime(TableTemp2['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp2 = TableTemp2.set_index('ds')
TableTemp2 = TableTemp2.sort_index()
TableTemp2 = TableTemp2.rename(columns={'H02':'Temp'})
######### TableTemp3 ##############
TableTemp3 = pd.pivot_table(TableTemp, values=['H03'], index=["ds"], aggfunc={'H03':'first'})
TableTemp3 = TableTemp3.reset_index()
TableTemp3['ds'] = TableTemp3['ds'] + '-' + '030000'
TableTemp3['ds'] = pd.to_datetime(TableTemp3['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp3 = TableTemp3.set_index('ds')
TableTemp3 = TableTemp3.sort_index()
TableTemp3 = TableTemp3.rename(columns={'H03':'Temp'})
######### TableTemp4 ##############
TableTemp4 = pd.pivot_table(TableTemp, values=['H04'], index=["ds"], aggfunc={'H04':'first'})
TableTemp4 = TableTemp4.reset_index()
TableTemp4['ds'] = TableTemp4['ds'] + '-' + '040000'
TableTemp4['ds'] = pd.to_datetime(TableTemp4['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp4 = TableTemp4.set_index('ds')
TableTemp4 = TableTemp4.sort_index()
TableTemp4 = TableTemp4.rename(columns={'H04':'Temp'})
######### TableTemp5 ##############
TableTemp5 = pd.pivot_table(TableTemp, values=['H05'], index=["ds"], aggfunc={'H05':'first'})
TableTemp5 = TableTemp5.reset_index()
TableTemp5['ds'] = TableTemp5['ds'] + '-' + '050000'
TableTemp5['ds'] = pd.to_datetime(TableTemp5['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp5 = TableTemp5.set_index('ds')
TableTemp5 = TableTemp5.sort_index()
TableTemp5 = TableTemp5.rename(columns={'H05':'Temp'})
######### TableTemp6 ##############
TableTemp6 = pd.pivot_table(TableTemp, values=['H06'], index=["ds"], aggfunc={'H06':'first'})
TableTemp6 = TableTemp6.reset_index()
TableTemp6['ds'] = TableTemp6['ds'] + '-' + '060000'
TableTemp6['ds'] = pd.to_datetime(TableTemp6['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp6 = TableTemp6.set_index('ds')
TableTemp6 = TableTemp6.sort_index()
TableTemp6 = TableTemp6.rename(columns={'H06':'Temp'})
######### TableTemp7 ##############
TableTemp7 = pd.pivot_table(TableTemp, values=['H07'], index=["ds"], aggfunc={'H07':'first'})
TableTemp7 = TableTemp7.reset_index()
TableTemp7['ds'] = TableTemp7['ds'] + '-' + '070000'
TableTemp7['ds'] = pd.to_datetime(TableTemp7['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp7 = TableTemp7.set_index('ds')
TableTemp7 = TableTemp7.sort_index()
TableTemp7 = TableTemp7.rename(columns={'H07':'Temp'})
######### TableTemp8 ##############
TableTemp8 = pd.pivot_table(TableTemp, values=['H08'], index=["ds"], aggfunc={'H08':'first'})
TableTemp8 = TableTemp8.reset_index()
TableTemp8['ds'] = TableTemp8['ds'] + '-' + '080000'
TableTemp8['ds'] = pd.to_datetime(TableTemp8['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp8 = TableTemp8.set_index('ds')
TableTemp8 = TableTemp8.sort_index()
TableTemp8 = TableTemp8.rename(columns={'H08':'Temp'})
######### TableTemp9 ##############
TableTemp9 = pd.pivot_table(TableTemp, values=['H09'], index=["ds"], aggfunc={'H09':'first'})
TableTemp9 = TableTemp9.reset_index()
TableTemp9['ds'] = TableTemp9['ds'] + '-' + '090000'
TableTemp9['ds'] = pd.to_datetime(TableTemp9['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp9 = TableTemp9.set_index('ds')
TableTemp9 = TableTemp9.sort_index()
TableTemp9 = TableTemp9.rename(columns={'H09':'Temp'})
######### TableTemp10 ##############
TableTemp10 = pd.pivot_table(TableTemp, values=['H10'], index=["ds"], aggfunc={'H10':'first'})
TableTemp10 = TableTemp10.reset_index()
TableTemp10['ds'] = TableTemp10['ds'] + '-' + '100000'
TableTemp10['ds'] = pd.to_datetime(TableTemp10['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp10 = TableTemp10.set_index('ds')
TableTemp10 = TableTemp10.sort_index()
TableTemp10 = TableTemp10.rename(columns={'H10':'Temp'})
######### TableTemp11 ##############
TableTemp11 = pd.pivot_table(TableTemp, values=['H11'], index=["ds"], aggfunc={'H11':'first'})
TableTemp11 = TableTemp11.reset_index()
TableTemp11['ds'] = TableTemp11['ds'] + '-' + '110000'
TableTemp11['ds'] = pd.to_datetime(TableTemp11['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp11 = TableTemp11.set_index('ds')
TableTemp11 = TableTemp11.sort_index()
TableTemp11 = TableTemp11.rename(columns={'H11':'Temp'})
######### TableTemp12 ##############
TableTemp12 = pd.pivot_table(TableTemp, values=['H12'], index=["ds"], aggfunc={'H12':'first'})
TableTemp12 = TableTemp12.reset_index()
TableTemp12['ds'] = TableTemp12['ds'] + '-' + '120000'
TableTemp12['ds'] = pd.to_datetime(TableTemp12['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp12 = TableTemp12.set_index('ds')
TableTemp12 = TableTemp12.sort_index()
TableTemp12 = TableTemp12.rename(columns={'H12':'Temp'})
######### TableTemp13 ##############
TableTemp13 = pd.pivot_table(TableTemp, values=['H13'], index=["ds"], aggfunc={'H13':'first'})
TableTemp13 = TableTemp13.reset_index()
TableTemp13['ds'] = TableTemp13['ds'] + '-' + '130000'
TableTemp13['ds'] = pd.to_datetime(TableTemp13['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp13 = TableTemp13.set_index('ds')
TableTemp13 = TableTemp13.sort_index()
TableTemp13 = TableTemp13.rename(columns={'H13':'Temp'})
######### TableTemp14 ##############
TableTemp14 = pd.pivot_table(TableTemp, values=['H14'], index=["ds"], aggfunc={'H14':'first'})
TableTemp14 = TableTemp14.reset_index()
TableTemp14['ds'] = TableTemp14['ds'] + '-' + '140000'
TableTemp14['ds'] = pd.to_datetime(TableTemp14['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp14 = TableTemp14.set_index('ds')
TableTemp14 = TableTemp14.sort_index()
TableTemp14 = TableTemp14.rename(columns={'H14':'Temp'})
######### TableTemp15 ##############
TableTemp15 = pd.pivot_table(TableTemp, values=['H15'], index=["ds"], aggfunc={'H15':'first'})
TableTemp15 = TableTemp15.reset_index()
TableTemp15['ds'] = TableTemp15['ds'] + '-' + '150000'
TableTemp15['ds'] = pd.to_datetime(TableTemp15['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp15 = TableTemp15.set_index('ds')
TableTemp15 = TableTemp15.sort_index()
TableTemp15 = TableTemp15.rename(columns={'H15':'Temp'})
######### TableTemp16 ##############
TableTemp16 = pd.pivot_table(TableTemp, values=['H16'], index=["ds"], aggfunc={'H16':'first'})
TableTemp16 = TableTemp16.reset_index()
TableTemp16['ds'] = TableTemp16['ds'] + '-' + '160000'
TableTemp16['ds'] = pd.to_datetime(TableTemp16['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp16 = TableTemp16.set_index('ds')
TableTemp16 = TableTemp16.sort_index()
TableTemp16 = TableTemp16.rename(columns={'H16':'Temp'})
######### TableTemp17 ##############
TableTemp17 = pd.pivot_table(TableTemp, values=['H17'], index=["ds"], aggfunc={'H17':'first'})
TableTemp17 = TableTemp17.reset_index()
TableTemp17['ds'] = TableTemp17['ds'] + '-' + '170000'
TableTemp17['ds'] = pd.to_datetime(TableTemp17['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp17 = TableTemp17.set_index('ds')
TableTemp17 = TableTemp17.sort_index()
TableTemp17 = TableTemp17.rename(columns={'H17':'Temp'})
######### TableTemp18 ##############
TableTemp18 = pd.pivot_table(TableTemp, values=['H18'], index=["ds"], aggfunc={'H18':'first'})
TableTemp18 = TableTemp18.reset_index()
TableTemp18['ds'] = TableTemp18['ds'] + '-' + '180000'
TableTemp18['ds'] = pd.to_datetime(TableTemp18['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp18 = TableTemp18.set_index('ds')
TableTemp18 = TableTemp18.sort_index()
TableTemp18 = TableTemp18.rename(columns={'H18':'Temp'})
######### TableTemp19 ##############
TableTemp19 = pd.pivot_table(TableTemp, values=['H19'], index=["ds"], aggfunc={'H19':'first'})
TableTemp19 = TableTemp19.reset_index()
TableTemp19['ds'] = TableTemp19['ds'] + '-' + '190000'
TableTemp19['ds'] = pd.to_datetime(TableTemp19['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp19 = TableTemp19.set_index('ds')
TableTemp19 = TableTemp19.sort_index()
TableTemp19 = TableTemp19.rename(columns={'H19':'Temp'})
######### TableTemp20 ##############
TableTemp20 = pd.pivot_table(TableTemp, values=['H20'], index=["ds"], aggfunc={'H20':'first'})
TableTemp20 = TableTemp20.reset_index()
TableTemp20['ds'] = TableTemp20['ds'] + '-' + '200000'
TableTemp20['ds'] = pd.to_datetime(TableTemp20['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp20 = TableTemp20.set_index('ds')
TableTemp20 = TableTemp20.sort_index()
TableTemp20 = TableTemp20.rename(columns={'H20':'Temp'})
######### TableTemp21 ##############
TableTemp21 = pd.pivot_table(TableTemp, values=['H21'], index=["ds"], aggfunc={'H21':'first'})
TableTemp21 = TableTemp21.reset_index()
TableTemp21['ds'] = TableTemp21['ds'] + '-' + '210000'
TableTemp21['ds'] = pd.to_datetime(TableTemp21['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp21 = TableTemp21.set_index('ds')
TableTemp21 = TableTemp21.sort_index()
TableTemp21 = TableTemp21.rename(columns={'H21':'Temp'})
######### TableTemp22 ##############
TableTemp22 = pd.pivot_table(TableTemp, values=['H22'], index=["ds"], aggfunc={'H22':'first'})
TableTemp22 = TableTemp22.reset_index()
TableTemp22['ds'] = TableTemp22['ds'] + '-' + '220000'
TableTemp22['ds'] = pd.to_datetime(TableTemp22['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp22 = TableTemp22.set_index('ds')
TableTemp22 = TableTemp22.sort_index()
TableTemp22 = TableTemp22.rename(columns={'H22':'Temp'})
######### TableTemp23 ##############
TableTemp23 = pd.pivot_table(TableTemp, values=['H23'], index=["ds"], aggfunc={'H23':'first'})
TableTemp23 = TableTemp23.reset_index()
TableTemp23['ds'] = TableTemp23['ds'] + '-' + '230000'
TableTemp23['ds'] = pd.to_datetime(TableTemp23['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp23 = TableTemp23.set_index('ds')
TableTemp23 = TableTemp23.sort_index()
TableTemp23 = TableTemp23.rename(columns={'H23':'Temp'})
######### TableTemp0 ##############
TableTemp0 = pd.pivot_table(TableTemp, values=['H24'], index=["ds"], aggfunc={'H24':'first'})
TableTemp0 = TableTemp0.reset_index()
TableTemp0['ds'] = TableTemp0['ds'] + '-' + '000000'
TableTemp0['ds'] = pd.to_datetime(TableTemp0['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp0 = TableTemp0.set_index('ds')
TableTemp0 = TableTemp0.sort_index()
TableTemp0 = TableTemp0.rename(columns={'H24':'Temp'})

# Concatenamos todas las tablas en una sola

In [12]:
Data_TempJun = pd.concat([TableTemp0,TableTemp1,TableTemp2,TableTemp3,TableTemp4,TableTemp5,TableTemp6,TableTemp7,TableTemp8,TableTemp9,TableTemp10,TableTemp11,TableTemp12,TableTemp13,TableTemp14,TableTemp15,TableTemp16,TableTemp17,TableTemp18,TableTemp19,TableTemp20,TableTemp21,TableTemp22,TableTemp23])
Data_TempJun = Data_TempJun.sort_index()
Data_TempJun.tail(5)

,Temp
ds,
2020-06-30 19:00:00,36.6
2020-06-30 20:00:00,35.4
2020-06-30 21:00:00,34.4
2020-06-30 22:00:00,33.0
2020-06-30 23:00:00,31.5


# 2.2) Procesamiento Datos Precipitación
## Obtenemos datos de Precipitación de la estación Hortaleza 

In [13]:
DataFiltrar2 = DataMeteo.loc[DataMeteo.loc[:,'ESTACION'] == 107]
DataPrec = DataFiltrar2.loc[DataFiltrar2.loc[:,'MAGNITUD'] == 89]
DataPrec = DataPrec.drop(columns=['PROVINCIA','MUNICIPIO','ESTACION','MAGNITUD','V01','V02','V03','V04','V05','V06','V07','V08','V09','V10','V11','V12','V13','V14','V15','V16','V17','V18','V19','V20','V21','V22','V23','V24'])
TablePrec = DataPrec
# Convertimos los elementos de las columnas "ANO" "MES" "DIA" a string
TablePrec['ANO']= TablePrec['ANO'].map(str)
TablePrec['MES']= TablePrec['MES'].map(str)
TablePrec['DIA']= TablePrec['DIA'].map(str)
# Ajustamos la columna ds con (-) para posteriormente obtener un timestamp
TablePrec['ds'] = TablePrec['ANO'] + '-' + TablePrec['MES'] + '-' + TablePrec['DIA']
TablePrec.tail(5)

,PUNTO_MUESTREO,ANO,MES,DIA,H01,H02,H03,H04,H05,H06,...,H16,H17,H18,H19,H20,H21,H22,H23,H24,ds
769,28079107_89_98,2020,6,26,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-6-26
770,28079107_89_98,2020,6,27,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-6-27
771,28079107_89_98,2020,6,28,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-6-28
772,28079107_89_98,2020,6,29,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-6-29
773,28079107_89_98,2020,6,30,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-6-30


In [14]:
######### TableTemp1 ##############
TablePrec1 = pd.pivot_table(TablePrec, values=['H01'], index=["ds"], aggfunc={'H01':'first'})
TablePrec1 = TablePrec1.reset_index()
TablePrec1['ds'] = TablePrec1['ds'] + '-' + '010000'
TablePrec1['ds'] = pd.to_datetime(TablePrec1['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec1 = TablePrec1.set_index('ds')
TablePrec1 = TablePrec1.sort_index()
TablePrec1 = TablePrec1.rename(columns={'H01':'Prec'})
######### TableTemp2 ##############
TablePrec2 = pd.pivot_table(TablePrec, values=['H02'], index=["ds"], aggfunc={'H02':'first'})
TablePrec2 = TablePrec2.reset_index()
TablePrec2['ds'] = TablePrec2['ds'] + '-' + '020000'
TablePrec2['ds'] = pd.to_datetime(TablePrec2['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec2 = TablePrec2.set_index('ds')
TablePrec2 = TablePrec2.sort_index()
TablePrec2 = TablePrec2.rename(columns={'H02':'Prec'})
######### TableTemp3 ##############
TablePrec3 = pd.pivot_table(TablePrec, values=['H03'], index=["ds"], aggfunc={'H03':'first'})
TablePrec3 = TablePrec3.reset_index()
TablePrec3['ds'] = TablePrec3['ds'] + '-' + '030000'
TablePrec3['ds'] = pd.to_datetime(TablePrec3['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec3 = TablePrec3.set_index('ds')
TablePrec3 = TablePrec3.sort_index()
TablePrec3 = TablePrec3.rename(columns={'H03':'Prec'})
######### TableTemp4 ##############
TablePrec4 = pd.pivot_table(TablePrec, values=['H04'], index=["ds"], aggfunc={'H04':'first'})
TablePrec4 = TablePrec4.reset_index()
TablePrec4['ds'] = TablePrec4['ds'] + '-' + '040000'
TablePrec4['ds'] = pd.to_datetime(TablePrec4['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec4 = TablePrec4.set_index('ds')
TablePrec4 = TablePrec4.sort_index()
TablePrec4 = TablePrec4.rename(columns={'H04':'Prec'})
######### TableTemp5 ##############
TablePrec5 = pd.pivot_table(TablePrec, values=['H05'], index=["ds"], aggfunc={'H05':'first'})
TablePrec5 = TablePrec5.reset_index()
TablePrec5['ds'] = TablePrec5['ds'] + '-' + '050000'
TablePrec5['ds'] = pd.to_datetime(TablePrec5['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec5 = TablePrec5.set_index('ds')
TablePrec5 = TablePrec5.sort_index()
TablePrec5 = TablePrec5.rename(columns={'H05':'Prec'})
######### TableTemp6 ##############
TablePrec6 = pd.pivot_table(TablePrec, values=['H06'], index=["ds"], aggfunc={'H06':'first'})
TablePrec6 = TablePrec6.reset_index()
TablePrec6['ds'] = TablePrec6['ds'] + '-' + '060000'
TablePrec6['ds'] = pd.to_datetime(TablePrec6['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec6 = TablePrec6.set_index('ds')
TablePrec6 = TablePrec6.sort_index()
TablePrec6 = TablePrec6.rename(columns={'H06':'Prec'})
######### TableTemp7 ##############
TablePrec7 = pd.pivot_table(TablePrec, values=['H07'], index=["ds"], aggfunc={'H07':'first'})
TablePrec7 = TablePrec7.reset_index()
TablePrec7['ds'] = TablePrec7['ds'] + '-' + '070000'
TablePrec7['ds'] = pd.to_datetime(TablePrec7['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec7 = TablePrec7.set_index('ds')
TablePrec7 = TablePrec7.sort_index()
TablePrec7 = TablePrec7.rename(columns={'H07':'Prec'})
######### TableTemp8 ##############
TablePrec8 = pd.pivot_table(TablePrec, values=['H08'], index=["ds"], aggfunc={'H08':'first'})
TablePrec8 = TablePrec8.reset_index()
TablePrec8['ds'] = TablePrec8['ds'] + '-' + '080000'
TablePrec8['ds'] = pd.to_datetime(TablePrec8['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec8 = TablePrec8.set_index('ds')
TablePrec8 = TablePrec8.sort_index()
TablePrec8 = TablePrec8.rename(columns={'H08':'Prec'})
######### TableTemp9 ##############
TablePrec9 = pd.pivot_table(TablePrec, values=['H09'], index=["ds"], aggfunc={'H09':'first'})
TablePrec9 = TablePrec9.reset_index()
TablePrec9['ds'] = TablePrec9['ds'] + '-' + '090000'
TablePrec9['ds'] = pd.to_datetime(TablePrec9['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec9 = TablePrec9.set_index('ds')
TablePrec9 = TablePrec9.sort_index()
TablePrec9 = TablePrec9.rename(columns={'H09':'Prec'})
######### TableTemp10 ##############
TablePrec10 = pd.pivot_table(TablePrec, values=['H10'], index=["ds"], aggfunc={'H10':'first'})
TablePrec10 = TablePrec10.reset_index()
TablePrec10['ds'] = TablePrec10['ds'] + '-' + '100000'
TablePrec10['ds'] = pd.to_datetime(TablePrec10['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec10 = TablePrec10.set_index('ds')
TablePrec10 = TablePrec10.sort_index()
TablePrec10 = TablePrec10.rename(columns={'H10':'Prec'})
######### TableTemp11 ##############
TablePrec11 = pd.pivot_table(TablePrec, values=['H11'], index=["ds"], aggfunc={'H11':'first'})
TablePrec11 = TablePrec11.reset_index()
TablePrec11['ds'] = TablePrec11['ds'] + '-' + '110000'
TablePrec11['ds'] = pd.to_datetime(TablePrec11['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec11 = TablePrec11.set_index('ds')
TablePrec11 = TablePrec11.sort_index()
TablePrec11 = TablePrec11.rename(columns={'H11':'Prec'})
######### TableTemp12 ##############
TablePrec12 = pd.pivot_table(TablePrec, values=['H12'], index=["ds"], aggfunc={'H12':'first'})
TablePrec12 = TablePrec12.reset_index()
TablePrec12['ds'] = TablePrec12['ds'] + '-' + '120000'
TablePrec12['ds'] = pd.to_datetime(TablePrec12['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec12 = TablePrec12.set_index('ds')
TablePrec12 = TablePrec12.sort_index()
TablePrec12 = TablePrec12.rename(columns={'H12':'Prec'})
######### TableTemp13 ##############
TablePrec13 = pd.pivot_table(TablePrec, values=['H13'], index=["ds"], aggfunc={'H13':'first'})
TablePrec13 = TablePrec13.reset_index()
TablePrec13['ds'] = TablePrec13['ds'] + '-' + '130000'
TablePrec13['ds'] = pd.to_datetime(TablePrec13['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec13 = TablePrec13.set_index('ds')
TablePrec13 = TablePrec13.sort_index()
TablePrec13 = TablePrec13.rename(columns={'H13':'Prec'})
######### TableTemp14 ##############
TablePrec14 = pd.pivot_table(TablePrec, values=['H14'], index=["ds"], aggfunc={'H14':'first'})
TablePrec14 = TablePrec14.reset_index()
TablePrec14['ds'] = TablePrec14['ds'] + '-' + '140000'
TablePrec14['ds'] = pd.to_datetime(TablePrec14['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec14 = TablePrec14.set_index('ds')
TablePrec14 = TablePrec14.sort_index()
TablePrec14 = TablePrec14.rename(columns={'H14':'Prec'})
######### TableTemp15 ##############
TablePrec15 = pd.pivot_table(TablePrec, values=['H15'], index=["ds"], aggfunc={'H15':'first'})
TablePrec15 = TablePrec15.reset_index()
TablePrec15['ds'] = TablePrec15['ds'] + '-' + '150000'
TablePrec15['ds'] = pd.to_datetime(TablePrec15['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec15 = TablePrec15.set_index('ds')
TablePrec15 = TablePrec15.sort_index()
TablePrec15 = TablePrec15.rename(columns={'H15':'Prec'})
######### TableTemp16 ##############
TablePrec16 = pd.pivot_table(TablePrec, values=['H16'], index=["ds"], aggfunc={'H16':'first'})
TablePrec16 = TablePrec16.reset_index()
TablePrec16['ds'] = TablePrec16['ds'] + '-' + '160000'
TablePrec16['ds'] = pd.to_datetime(TablePrec16['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec16 = TablePrec16.set_index('ds')
TablePrec16 = TablePrec16.sort_index()
TablePrec16 = TablePrec16.rename(columns={'H16':'Prec'})
######### TableTemp17 ##############
TablePrec17 = pd.pivot_table(TablePrec, values=['H17'], index=["ds"], aggfunc={'H17':'first'})
TablePrec17 = TablePrec17.reset_index()
TablePrec17['ds'] = TablePrec17['ds'] + '-' + '170000'
TablePrec17['ds'] = pd.to_datetime(TablePrec17['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec17 = TablePrec17.set_index('ds')
TablePrec17 = TablePrec17.sort_index()
TablePrec17 = TablePrec17.rename(columns={'H17':'Prec'})
######### TableTemp18 ##############
TablePrec18 = pd.pivot_table(TablePrec, values=['H18'], index=["ds"], aggfunc={'H18':'first'})
TablePrec18 = TablePrec18.reset_index()
TablePrec18['ds'] = TablePrec18['ds'] + '-' + '180000'
TablePrec18['ds'] = pd.to_datetime(TablePrec18['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec18 = TablePrec18.set_index('ds')
TablePrec18 = TablePrec18.sort_index()
TablePrec18 = TablePrec18.rename(columns={'H18':'Prec'})
######### TableTemp19 ##############
TablePrec19 = pd.pivot_table(TablePrec, values=['H19'], index=["ds"], aggfunc={'H19':'first'})
TablePrec19 = TablePrec19.reset_index()
TablePrec19['ds'] = TablePrec19['ds'] + '-' + '190000'
TablePrec19['ds'] = pd.to_datetime(TablePrec19['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec19 = TablePrec19.set_index('ds')
TablePrec19 = TablePrec19.sort_index()
TablePrec19 = TablePrec19.rename(columns={'H19':'Prec'})
######### TableTemp20 ##############
TablePrec20 = pd.pivot_table(TablePrec, values=['H20'], index=["ds"], aggfunc={'H20':'first'})
TablePrec20 = TablePrec20.reset_index()
TablePrec20['ds'] = TablePrec20['ds'] + '-' + '200000'
TablePrec20['ds'] = pd.to_datetime(TablePrec20['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec20 = TablePrec20.set_index('ds')
TablePrec20 = TablePrec20.sort_index()
TablePrec20 = TablePrec20.rename(columns={'H20':'Prec'})
######### TableTemp21 ##############
TablePrec21 = pd.pivot_table(TablePrec, values=['H21'], index=["ds"], aggfunc={'H21':'first'})
TablePrec21 = TablePrec21.reset_index()
TablePrec21['ds'] = TablePrec21['ds'] + '-' + '210000'
TablePrec21['ds'] = pd.to_datetime(TablePrec21['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec21 = TablePrec21.set_index('ds')
TablePrec21 = TablePrec21.sort_index()
TablePrec21 = TablePrec21.rename(columns={'H21':'Prec'})
######### TableTemp22 ##############
TablePrec22 = pd.pivot_table(TablePrec, values=['H22'], index=["ds"], aggfunc={'H22':'first'})
TablePrec22 = TablePrec22.reset_index()
TablePrec22['ds'] = TablePrec22['ds'] + '-' + '220000'
TablePrec22['ds'] = pd.to_datetime(TablePrec22['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec22 = TablePrec22.set_index('ds')
TablePrec22 = TablePrec22.sort_index()
TablePrec22 = TablePrec22.rename(columns={'H22':'Prec'})
######### TableTemp23 ##############
TablePrec23 = pd.pivot_table(TablePrec, values=['H23'], index=["ds"], aggfunc={'H23':'first'})
TablePrec23 = TablePrec23.reset_index()
TablePrec23['ds'] = TablePrec23['ds'] + '-' + '230000'
TablePrec23['ds'] = pd.to_datetime(TablePrec23['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec23 = TablePrec23.set_index('ds')
TablePrec23 = TablePrec23.sort_index()
TablePrec23 = TablePrec23.rename(columns={'H23':'Prec'})
######### TableTemp0 ##############
TablePrec0 = pd.pivot_table(TablePrec, values=['H24'], index=["ds"], aggfunc={'H24':'first'})
TablePrec0 = TablePrec0.reset_index()
TablePrec0['ds'] = TablePrec0['ds'] + '-' + '000000'
TablePrec0['ds'] = pd.to_datetime(TablePrec0['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec0 = TablePrec0.set_index('ds')
TablePrec0 = TablePrec0.sort_index()
TablePrec0 = TablePrec0.rename(columns={'H24':'Prec'})

Data_PrecJun = pd.concat([TablePrec0,TablePrec1,TablePrec2,TablePrec3,TablePrec4,TablePrec5,TablePrec6,TablePrec7,TablePrec8,TablePrec9,TablePrec10,TablePrec11,TablePrec12,TablePrec13,TablePrec14,TablePrec15,TablePrec16,TablePrec17,TablePrec18,TablePrec19,TablePrec20,TablePrec21,TablePrec22,TablePrec23])
Data_PrecJun = Data_PrecJun.sort_index()
Data_PrecJun.tail(5)

,Prec
ds,
2020-06-30 19:00:00,0.0
2020-06-30 20:00:00,0.0
2020-06-30 21:00:00,0.0
2020-06-30 22:00:00,0.0
2020-06-30 23:00:00,0.0


# 3) Datos Metereologicos Julio

In [15]:
DataMeteo = pd.read_csv("/home/jonathan/tesis/Data_Temperatura_Lluvia/jul_meteo20.csv", sep=';')
DataMeteo.head(1)

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H01,V01,...,H20,V20,H21,V21,H22,V22,H23,V23,H24,V24
0,28,79,102,81,28079102_81_98,2020,7,1,1.13,V,...,2.57,V,3.3,V,3.3,V,3.03,V,2.67,V


# 3.1) Procesamiento Datos de Temperatura
## Obtenemos datos de temperatura de la estación Chamberí

In [16]:
DataFiltrar = DataMeteo.loc[DataMeteo.loc[:,'ESTACION'] == 109]
DataTemp = DataFiltrar.loc[DataFiltrar.loc[:,'MAGNITUD'] == 83]
DataTemp = DataTemp.drop(columns=['PROVINCIA','MUNICIPIO','ESTACION','MAGNITUD','V01','V02','V03','V04','V05','V06','V07','V08','V09','V10','V11','V12','V13','V14','V15','V16','V17','V18','V19','V20','V21','V22','V23','V24'])
TableTemp = DataTemp
# Convertimos los elementos de las columnas "ANO" "MES" "DIA" a string
TableTemp['ANO']= TableTemp['ANO'].map(str)
TableTemp['MES']= TableTemp['MES'].map(str)
TableTemp['DIA']= TableTemp['DIA'].map(str)
# Ajustamos la columna ds con (-) para posteriormente obtener un timestamp
TableTemp['ds'] = TableTemp['ANO'] + '-' + TableTemp['MES'] + '-' + TableTemp['DIA']
TableTemp.tail(5)

,PUNTO_MUESTREO,ANO,MES,DIA,H01,H02,H03,H04,H05,H06,...,H16,H17,H18,H19,H20,H21,H22,H23,H24,ds
956,28079109_83_98,2020,7,27,31.7,30.6,29.4,28.6,27.8,27.2,...,38.9,39.6,40.0,39.6,38.5,37.6,36.3,34.7,33.4,2020-7-27
957,28079109_83_98,2020,7,28,32.2,31.9,30.6,29.5,28.9,27.9,...,35.7,36.2,36.2,35.9,35.0,34.4,33.3,32.6,31.5,2020-7-28
958,28079109_83_98,2020,7,29,30.9,29.7,28.7,27.9,27.6,26.9,...,38.0,38.4,38.4,38.2,36.7,35.9,35.1,34.2,32.6,2020-7-29
959,28079109_83_98,2020,7,30,31.1,30.1,29.3,28.5,27.7,26.5,...,38.4,38.6,38.6,36.6,36.8,36.2,34.5,32.7,31.7,2020-7-30
960,28079109_83_98,2020,7,31,30.8,30.4,29.1,27.8,26.8,26.4,...,38.1,38.4,38.9,38.3,37.1,36.0,34.6,33.3,32.2,2020-7-31


# Obtenemos tablas de las 24 horas TableTemp#

In [17]:
######### TableTemp1 ##############
TableTemp1 = pd.pivot_table(TableTemp, values=['H01'], index=["ds"], aggfunc={'H01':'first'})
TableTemp1 = TableTemp1.reset_index()
TableTemp1['ds'] = TableTemp1['ds'] + '-' + '010000'
TableTemp1['ds'] = pd.to_datetime(TableTemp1['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp1 = TableTemp1.set_index('ds')
TableTemp1 = TableTemp1.sort_index()
TableTemp1 = TableTemp1.rename(columns={'H01':'Temp'})
######### TableTemp2 ##############
TableTemp2 = pd.pivot_table(TableTemp, values=['H02'], index=["ds"], aggfunc={'H02':'first'})
TableTemp2 = TableTemp2.reset_index()
TableTemp2['ds'] = TableTemp2['ds'] + '-' + '020000'
TableTemp2['ds'] = pd.to_datetime(TableTemp2['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp2 = TableTemp2.set_index('ds')
TableTemp2 = TableTemp2.sort_index()
TableTemp2 = TableTemp2.rename(columns={'H02':'Temp'})
######### TableTemp3 ##############
TableTemp3 = pd.pivot_table(TableTemp, values=['H03'], index=["ds"], aggfunc={'H03':'first'})
TableTemp3 = TableTemp3.reset_index()
TableTemp3['ds'] = TableTemp3['ds'] + '-' + '030000'
TableTemp3['ds'] = pd.to_datetime(TableTemp3['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp3 = TableTemp3.set_index('ds')
TableTemp3 = TableTemp3.sort_index()
TableTemp3 = TableTemp3.rename(columns={'H03':'Temp'})
######### TableTemp4 ##############
TableTemp4 = pd.pivot_table(TableTemp, values=['H04'], index=["ds"], aggfunc={'H04':'first'})
TableTemp4 = TableTemp4.reset_index()
TableTemp4['ds'] = TableTemp4['ds'] + '-' + '040000'
TableTemp4['ds'] = pd.to_datetime(TableTemp4['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp4 = TableTemp4.set_index('ds')
TableTemp4 = TableTemp4.sort_index()
TableTemp4 = TableTemp4.rename(columns={'H04':'Temp'})
######### TableTemp5 ##############
TableTemp5 = pd.pivot_table(TableTemp, values=['H05'], index=["ds"], aggfunc={'H05':'first'})
TableTemp5 = TableTemp5.reset_index()
TableTemp5['ds'] = TableTemp5['ds'] + '-' + '050000'
TableTemp5['ds'] = pd.to_datetime(TableTemp5['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp5 = TableTemp5.set_index('ds')
TableTemp5 = TableTemp5.sort_index()
TableTemp5 = TableTemp5.rename(columns={'H05':'Temp'})
######### TableTemp6 ##############
TableTemp6 = pd.pivot_table(TableTemp, values=['H06'], index=["ds"], aggfunc={'H06':'first'})
TableTemp6 = TableTemp6.reset_index()
TableTemp6['ds'] = TableTemp6['ds'] + '-' + '060000'
TableTemp6['ds'] = pd.to_datetime(TableTemp6['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp6 = TableTemp6.set_index('ds')
TableTemp6 = TableTemp6.sort_index()
TableTemp6 = TableTemp6.rename(columns={'H06':'Temp'})
######### TableTemp7 ##############
TableTemp7 = pd.pivot_table(TableTemp, values=['H07'], index=["ds"], aggfunc={'H07':'first'})
TableTemp7 = TableTemp7.reset_index()
TableTemp7['ds'] = TableTemp7['ds'] + '-' + '070000'
TableTemp7['ds'] = pd.to_datetime(TableTemp7['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp7 = TableTemp7.set_index('ds')
TableTemp7 = TableTemp7.sort_index()
TableTemp7 = TableTemp7.rename(columns={'H07':'Temp'})
######### TableTemp8 ##############
TableTemp8 = pd.pivot_table(TableTemp, values=['H08'], index=["ds"], aggfunc={'H08':'first'})
TableTemp8 = TableTemp8.reset_index()
TableTemp8['ds'] = TableTemp8['ds'] + '-' + '080000'
TableTemp8['ds'] = pd.to_datetime(TableTemp8['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp8 = TableTemp8.set_index('ds')
TableTemp8 = TableTemp8.sort_index()
TableTemp8 = TableTemp8.rename(columns={'H08':'Temp'})
######### TableTemp9 ##############
TableTemp9 = pd.pivot_table(TableTemp, values=['H09'], index=["ds"], aggfunc={'H09':'first'})
TableTemp9 = TableTemp9.reset_index()
TableTemp9['ds'] = TableTemp9['ds'] + '-' + '090000'
TableTemp9['ds'] = pd.to_datetime(TableTemp9['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp9 = TableTemp9.set_index('ds')
TableTemp9 = TableTemp9.sort_index()
TableTemp9 = TableTemp9.rename(columns={'H09':'Temp'})
######### TableTemp10 ##############
TableTemp10 = pd.pivot_table(TableTemp, values=['H10'], index=["ds"], aggfunc={'H10':'first'})
TableTemp10 = TableTemp10.reset_index()
TableTemp10['ds'] = TableTemp10['ds'] + '-' + '100000'
TableTemp10['ds'] = pd.to_datetime(TableTemp10['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp10 = TableTemp10.set_index('ds')
TableTemp10 = TableTemp10.sort_index()
TableTemp10 = TableTemp10.rename(columns={'H10':'Temp'})
######### TableTemp11 ##############
TableTemp11 = pd.pivot_table(TableTemp, values=['H11'], index=["ds"], aggfunc={'H11':'first'})
TableTemp11 = TableTemp11.reset_index()
TableTemp11['ds'] = TableTemp11['ds'] + '-' + '110000'
TableTemp11['ds'] = pd.to_datetime(TableTemp11['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp11 = TableTemp11.set_index('ds')
TableTemp11 = TableTemp11.sort_index()
TableTemp11 = TableTemp11.rename(columns={'H11':'Temp'})
######### TableTemp12 ##############
TableTemp12 = pd.pivot_table(TableTemp, values=['H12'], index=["ds"], aggfunc={'H12':'first'})
TableTemp12 = TableTemp12.reset_index()
TableTemp12['ds'] = TableTemp12['ds'] + '-' + '120000'
TableTemp12['ds'] = pd.to_datetime(TableTemp12['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp12 = TableTemp12.set_index('ds')
TableTemp12 = TableTemp12.sort_index()
TableTemp12 = TableTemp12.rename(columns={'H12':'Temp'})
######### TableTemp13 ##############
TableTemp13 = pd.pivot_table(TableTemp, values=['H13'], index=["ds"], aggfunc={'H13':'first'})
TableTemp13 = TableTemp13.reset_index()
TableTemp13['ds'] = TableTemp13['ds'] + '-' + '130000'
TableTemp13['ds'] = pd.to_datetime(TableTemp13['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp13 = TableTemp13.set_index('ds')
TableTemp13 = TableTemp13.sort_index()
TableTemp13 = TableTemp13.rename(columns={'H13':'Temp'})
######### TableTemp14 ##############
TableTemp14 = pd.pivot_table(TableTemp, values=['H14'], index=["ds"], aggfunc={'H14':'first'})
TableTemp14 = TableTemp14.reset_index()
TableTemp14['ds'] = TableTemp14['ds'] + '-' + '140000'
TableTemp14['ds'] = pd.to_datetime(TableTemp14['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp14 = TableTemp14.set_index('ds')
TableTemp14 = TableTemp14.sort_index()
TableTemp14 = TableTemp14.rename(columns={'H14':'Temp'})
######### TableTemp15 ##############
TableTemp15 = pd.pivot_table(TableTemp, values=['H15'], index=["ds"], aggfunc={'H15':'first'})
TableTemp15 = TableTemp15.reset_index()
TableTemp15['ds'] = TableTemp15['ds'] + '-' + '150000'
TableTemp15['ds'] = pd.to_datetime(TableTemp15['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp15 = TableTemp15.set_index('ds')
TableTemp15 = TableTemp15.sort_index()
TableTemp15 = TableTemp15.rename(columns={'H15':'Temp'})
######### TableTemp16 ##############
TableTemp16 = pd.pivot_table(TableTemp, values=['H16'], index=["ds"], aggfunc={'H16':'first'})
TableTemp16 = TableTemp16.reset_index()
TableTemp16['ds'] = TableTemp16['ds'] + '-' + '160000'
TableTemp16['ds'] = pd.to_datetime(TableTemp16['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp16 = TableTemp16.set_index('ds')
TableTemp16 = TableTemp16.sort_index()
TableTemp16 = TableTemp16.rename(columns={'H16':'Temp'})
######### TableTemp17 ##############
TableTemp17 = pd.pivot_table(TableTemp, values=['H17'], index=["ds"], aggfunc={'H17':'first'})
TableTemp17 = TableTemp17.reset_index()
TableTemp17['ds'] = TableTemp17['ds'] + '-' + '170000'
TableTemp17['ds'] = pd.to_datetime(TableTemp17['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp17 = TableTemp17.set_index('ds')
TableTemp17 = TableTemp17.sort_index()
TableTemp17 = TableTemp17.rename(columns={'H17':'Temp'})
######### TableTemp18 ##############
TableTemp18 = pd.pivot_table(TableTemp, values=['H18'], index=["ds"], aggfunc={'H18':'first'})
TableTemp18 = TableTemp18.reset_index()
TableTemp18['ds'] = TableTemp18['ds'] + '-' + '180000'
TableTemp18['ds'] = pd.to_datetime(TableTemp18['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp18 = TableTemp18.set_index('ds')
TableTemp18 = TableTemp18.sort_index()
TableTemp18 = TableTemp18.rename(columns={'H18':'Temp'})
######### TableTemp19 ##############
TableTemp19 = pd.pivot_table(TableTemp, values=['H19'], index=["ds"], aggfunc={'H19':'first'})
TableTemp19 = TableTemp19.reset_index()
TableTemp19['ds'] = TableTemp19['ds'] + '-' + '190000'
TableTemp19['ds'] = pd.to_datetime(TableTemp19['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp19 = TableTemp19.set_index('ds')
TableTemp19 = TableTemp19.sort_index()
TableTemp19 = TableTemp19.rename(columns={'H19':'Temp'})
######### TableTemp20 ##############
TableTemp20 = pd.pivot_table(TableTemp, values=['H20'], index=["ds"], aggfunc={'H20':'first'})
TableTemp20 = TableTemp20.reset_index()
TableTemp20['ds'] = TableTemp20['ds'] + '-' + '200000'
TableTemp20['ds'] = pd.to_datetime(TableTemp20['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp20 = TableTemp20.set_index('ds')
TableTemp20 = TableTemp20.sort_index()
TableTemp20 = TableTemp20.rename(columns={'H20':'Temp'})
######### TableTemp21 ##############
TableTemp21 = pd.pivot_table(TableTemp, values=['H21'], index=["ds"], aggfunc={'H21':'first'})
TableTemp21 = TableTemp21.reset_index()
TableTemp21['ds'] = TableTemp21['ds'] + '-' + '210000'
TableTemp21['ds'] = pd.to_datetime(TableTemp21['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp21 = TableTemp21.set_index('ds')
TableTemp21 = TableTemp21.sort_index()
TableTemp21 = TableTemp21.rename(columns={'H21':'Temp'})
######### TableTemp22 ##############
TableTemp22 = pd.pivot_table(TableTemp, values=['H22'], index=["ds"], aggfunc={'H22':'first'})
TableTemp22 = TableTemp22.reset_index()
TableTemp22['ds'] = TableTemp22['ds'] + '-' + '220000'
TableTemp22['ds'] = pd.to_datetime(TableTemp22['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp22 = TableTemp22.set_index('ds')
TableTemp22 = TableTemp22.sort_index()
TableTemp22 = TableTemp22.rename(columns={'H22':'Temp'})
######### TableTemp23 ##############
TableTemp23 = pd.pivot_table(TableTemp, values=['H23'], index=["ds"], aggfunc={'H23':'first'})
TableTemp23 = TableTemp23.reset_index()
TableTemp23['ds'] = TableTemp23['ds'] + '-' + '230000'
TableTemp23['ds'] = pd.to_datetime(TableTemp23['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp23 = TableTemp23.set_index('ds')
TableTemp23 = TableTemp23.sort_index()
TableTemp23 = TableTemp23.rename(columns={'H23':'Temp'})
######### TableTemp0 ##############
TableTemp0 = pd.pivot_table(TableTemp, values=['H24'], index=["ds"], aggfunc={'H24':'first'})
TableTemp0 = TableTemp0.reset_index()
TableTemp0['ds'] = TableTemp0['ds'] + '-' + '000000'
TableTemp0['ds'] = pd.to_datetime(TableTemp0['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp0 = TableTemp0.set_index('ds')
TableTemp0 = TableTemp0.sort_index()
TableTemp0 = TableTemp0.rename(columns={'H24':'Temp'})

# Concatenamos todas las tablas en una sola

In [18]:
Data_TempJul = pd.concat([TableTemp0,TableTemp1,TableTemp2,TableTemp3,TableTemp4,TableTemp5,TableTemp6,TableTemp7,TableTemp8,TableTemp9,TableTemp10,TableTemp11,TableTemp12,TableTemp13,TableTemp14,TableTemp15,TableTemp16,TableTemp17,TableTemp18,TableTemp19,TableTemp20,TableTemp21,TableTemp22,TableTemp23])
Data_TempJul = Data_TempJul.sort_index()
Data_TempJul.tail(5)

,Temp
ds,
2020-07-31 19:00:00,38.3
2020-07-31 20:00:00,37.1
2020-07-31 21:00:00,36.0
2020-07-31 22:00:00,34.6
2020-07-31 23:00:00,33.3


# 3.2) Procesamiento Datos Precipitación
## Obtenemos datos de Precipitación de la estación Hortaleza 

In [19]:
DataFiltrar2 = DataMeteo.loc[DataMeteo.loc[:,'ESTACION'] == 107]
DataPrec = DataFiltrar2.loc[DataFiltrar2.loc[:,'MAGNITUD'] == 89]
DataPrec = DataPrec.drop(columns=['PROVINCIA','MUNICIPIO','ESTACION','MAGNITUD','V01','V02','V03','V04','V05','V06','V07','V08','V09','V10','V11','V12','V13','V14','V15','V16','V17','V18','V19','V20','V21','V22','V23','V24'])
TablePrec = DataPrec
# Convertimos los elementos de las columnas "ANO" "MES" "DIA" a string
TablePrec['ANO']= TablePrec['ANO'].map(str)
TablePrec['MES']= TablePrec['MES'].map(str)
TablePrec['DIA']= TablePrec['DIA'].map(str)
# Ajustamos la columna ds con (-) para posteriormente obtener un timestamp
TablePrec['ds'] = TablePrec['ANO'] + '-' + TablePrec['MES'] + '-' + TablePrec['DIA']
TablePrec.tail(5)

,PUNTO_MUESTREO,ANO,MES,DIA,H01,H02,H03,H04,H05,H06,...,H16,H17,H18,H19,H20,H21,H22,H23,H24,ds
708,28079107_89_98,2020,7,27,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-7-27
709,28079107_89_98,2020,7,28,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-7-28
710,28079107_89_98,2020,7,29,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-7-29
711,28079107_89_98,2020,7,30,0.0,0.0,0.0,0.0,0.0,0.0,...,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-7-30
712,28079107_89_98,2020,7,31,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-7-31


In [20]:
######### TableTemp1 ##############
TablePrec1 = pd.pivot_table(TablePrec, values=['H01'], index=["ds"], aggfunc={'H01':'first'})
TablePrec1 = TablePrec1.reset_index()
TablePrec1['ds'] = TablePrec1['ds'] + '-' + '010000'
TablePrec1['ds'] = pd.to_datetime(TablePrec1['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec1 = TablePrec1.set_index('ds')
TablePrec1 = TablePrec1.sort_index()
TablePrec1 = TablePrec1.rename(columns={'H01':'Prec'})
######### TableTemp2 ##############
TablePrec2 = pd.pivot_table(TablePrec, values=['H02'], index=["ds"], aggfunc={'H02':'first'})
TablePrec2 = TablePrec2.reset_index()
TablePrec2['ds'] = TablePrec2['ds'] + '-' + '020000'
TablePrec2['ds'] = pd.to_datetime(TablePrec2['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec2 = TablePrec2.set_index('ds')
TablePrec2 = TablePrec2.sort_index()
TablePrec2 = TablePrec2.rename(columns={'H02':'Prec'})
######### TableTemp3 ##############
TablePrec3 = pd.pivot_table(TablePrec, values=['H03'], index=["ds"], aggfunc={'H03':'first'})
TablePrec3 = TablePrec3.reset_index()
TablePrec3['ds'] = TablePrec3['ds'] + '-' + '030000'
TablePrec3['ds'] = pd.to_datetime(TablePrec3['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec3 = TablePrec3.set_index('ds')
TablePrec3 = TablePrec3.sort_index()
TablePrec3 = TablePrec3.rename(columns={'H03':'Prec'})
######### TableTemp4 ##############
TablePrec4 = pd.pivot_table(TablePrec, values=['H04'], index=["ds"], aggfunc={'H04':'first'})
TablePrec4 = TablePrec4.reset_index()
TablePrec4['ds'] = TablePrec4['ds'] + '-' + '040000'
TablePrec4['ds'] = pd.to_datetime(TablePrec4['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec4 = TablePrec4.set_index('ds')
TablePrec4 = TablePrec4.sort_index()
TablePrec4 = TablePrec4.rename(columns={'H04':'Prec'})
######### TableTemp5 ##############
TablePrec5 = pd.pivot_table(TablePrec, values=['H05'], index=["ds"], aggfunc={'H05':'first'})
TablePrec5 = TablePrec5.reset_index()
TablePrec5['ds'] = TablePrec5['ds'] + '-' + '050000'
TablePrec5['ds'] = pd.to_datetime(TablePrec5['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec5 = TablePrec5.set_index('ds')
TablePrec5 = TablePrec5.sort_index()
TablePrec5 = TablePrec5.rename(columns={'H05':'Prec'})
######### TableTemp6 ##############
TablePrec6 = pd.pivot_table(TablePrec, values=['H06'], index=["ds"], aggfunc={'H06':'first'})
TablePrec6 = TablePrec6.reset_index()
TablePrec6['ds'] = TablePrec6['ds'] + '-' + '060000'
TablePrec6['ds'] = pd.to_datetime(TablePrec6['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec6 = TablePrec6.set_index('ds')
TablePrec6 = TablePrec6.sort_index()
TablePrec6 = TablePrec6.rename(columns={'H06':'Prec'})
######### TableTemp7 ##############
TablePrec7 = pd.pivot_table(TablePrec, values=['H07'], index=["ds"], aggfunc={'H07':'first'})
TablePrec7 = TablePrec7.reset_index()
TablePrec7['ds'] = TablePrec7['ds'] + '-' + '070000'
TablePrec7['ds'] = pd.to_datetime(TablePrec7['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec7 = TablePrec7.set_index('ds')
TablePrec7 = TablePrec7.sort_index()
TablePrec7 = TablePrec7.rename(columns={'H07':'Prec'})
######### TableTemp8 ##############
TablePrec8 = pd.pivot_table(TablePrec, values=['H08'], index=["ds"], aggfunc={'H08':'first'})
TablePrec8 = TablePrec8.reset_index()
TablePrec8['ds'] = TablePrec8['ds'] + '-' + '080000'
TablePrec8['ds'] = pd.to_datetime(TablePrec8['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec8 = TablePrec8.set_index('ds')
TablePrec8 = TablePrec8.sort_index()
TablePrec8 = TablePrec8.rename(columns={'H08':'Prec'})
######### TableTemp9 ##############
TablePrec9 = pd.pivot_table(TablePrec, values=['H09'], index=["ds"], aggfunc={'H09':'first'})
TablePrec9 = TablePrec9.reset_index()
TablePrec9['ds'] = TablePrec9['ds'] + '-' + '090000'
TablePrec9['ds'] = pd.to_datetime(TablePrec9['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec9 = TablePrec9.set_index('ds')
TablePrec9 = TablePrec9.sort_index()
TablePrec9 = TablePrec9.rename(columns={'H09':'Prec'})
######### TableTemp10 ##############
TablePrec10 = pd.pivot_table(TablePrec, values=['H10'], index=["ds"], aggfunc={'H10':'first'})
TablePrec10 = TablePrec10.reset_index()
TablePrec10['ds'] = TablePrec10['ds'] + '-' + '100000'
TablePrec10['ds'] = pd.to_datetime(TablePrec10['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec10 = TablePrec10.set_index('ds')
TablePrec10 = TablePrec10.sort_index()
TablePrec10 = TablePrec10.rename(columns={'H10':'Prec'})
######### TableTemp11 ##############
TablePrec11 = pd.pivot_table(TablePrec, values=['H11'], index=["ds"], aggfunc={'H11':'first'})
TablePrec11 = TablePrec11.reset_index()
TablePrec11['ds'] = TablePrec11['ds'] + '-' + '110000'
TablePrec11['ds'] = pd.to_datetime(TablePrec11['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec11 = TablePrec11.set_index('ds')
TablePrec11 = TablePrec11.sort_index()
TablePrec11 = TablePrec11.rename(columns={'H11':'Prec'})
######### TableTemp12 ##############
TablePrec12 = pd.pivot_table(TablePrec, values=['H12'], index=["ds"], aggfunc={'H12':'first'})
TablePrec12 = TablePrec12.reset_index()
TablePrec12['ds'] = TablePrec12['ds'] + '-' + '120000'
TablePrec12['ds'] = pd.to_datetime(TablePrec12['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec12 = TablePrec12.set_index('ds')
TablePrec12 = TablePrec12.sort_index()
TablePrec12 = TablePrec12.rename(columns={'H12':'Prec'})
######### TableTemp13 ##############
TablePrec13 = pd.pivot_table(TablePrec, values=['H13'], index=["ds"], aggfunc={'H13':'first'})
TablePrec13 = TablePrec13.reset_index()
TablePrec13['ds'] = TablePrec13['ds'] + '-' + '130000'
TablePrec13['ds'] = pd.to_datetime(TablePrec13['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec13 = TablePrec13.set_index('ds')
TablePrec13 = TablePrec13.sort_index()
TablePrec13 = TablePrec13.rename(columns={'H13':'Prec'})
######### TableTemp14 ##############
TablePrec14 = pd.pivot_table(TablePrec, values=['H14'], index=["ds"], aggfunc={'H14':'first'})
TablePrec14 = TablePrec14.reset_index()
TablePrec14['ds'] = TablePrec14['ds'] + '-' + '140000'
TablePrec14['ds'] = pd.to_datetime(TablePrec14['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec14 = TablePrec14.set_index('ds')
TablePrec14 = TablePrec14.sort_index()
TablePrec14 = TablePrec14.rename(columns={'H14':'Prec'})
######### TableTemp15 ##############
TablePrec15 = pd.pivot_table(TablePrec, values=['H15'], index=["ds"], aggfunc={'H15':'first'})
TablePrec15 = TablePrec15.reset_index()
TablePrec15['ds'] = TablePrec15['ds'] + '-' + '150000'
TablePrec15['ds'] = pd.to_datetime(TablePrec15['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec15 = TablePrec15.set_index('ds')
TablePrec15 = TablePrec15.sort_index()
TablePrec15 = TablePrec15.rename(columns={'H15':'Prec'})
######### TableTemp16 ##############
TablePrec16 = pd.pivot_table(TablePrec, values=['H16'], index=["ds"], aggfunc={'H16':'first'})
TablePrec16 = TablePrec16.reset_index()
TablePrec16['ds'] = TablePrec16['ds'] + '-' + '160000'
TablePrec16['ds'] = pd.to_datetime(TablePrec16['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec16 = TablePrec16.set_index('ds')
TablePrec16 = TablePrec16.sort_index()
TablePrec16 = TablePrec16.rename(columns={'H16':'Prec'})
######### TableTemp17 ##############
TablePrec17 = pd.pivot_table(TablePrec, values=['H17'], index=["ds"], aggfunc={'H17':'first'})
TablePrec17 = TablePrec17.reset_index()
TablePrec17['ds'] = TablePrec17['ds'] + '-' + '170000'
TablePrec17['ds'] = pd.to_datetime(TablePrec17['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec17 = TablePrec17.set_index('ds')
TablePrec17 = TablePrec17.sort_index()
TablePrec17 = TablePrec17.rename(columns={'H17':'Prec'})
######### TableTemp18 ##############
TablePrec18 = pd.pivot_table(TablePrec, values=['H18'], index=["ds"], aggfunc={'H18':'first'})
TablePrec18 = TablePrec18.reset_index()
TablePrec18['ds'] = TablePrec18['ds'] + '-' + '180000'
TablePrec18['ds'] = pd.to_datetime(TablePrec18['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec18 = TablePrec18.set_index('ds')
TablePrec18 = TablePrec18.sort_index()
TablePrec18 = TablePrec18.rename(columns={'H18':'Prec'})
######### TableTemp19 ##############
TablePrec19 = pd.pivot_table(TablePrec, values=['H19'], index=["ds"], aggfunc={'H19':'first'})
TablePrec19 = TablePrec19.reset_index()
TablePrec19['ds'] = TablePrec19['ds'] + '-' + '190000'
TablePrec19['ds'] = pd.to_datetime(TablePrec19['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec19 = TablePrec19.set_index('ds')
TablePrec19 = TablePrec19.sort_index()
TablePrec19 = TablePrec19.rename(columns={'H19':'Prec'})
######### TableTemp20 ##############
TablePrec20 = pd.pivot_table(TablePrec, values=['H20'], index=["ds"], aggfunc={'H20':'first'})
TablePrec20 = TablePrec20.reset_index()
TablePrec20['ds'] = TablePrec20['ds'] + '-' + '200000'
TablePrec20['ds'] = pd.to_datetime(TablePrec20['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec20 = TablePrec20.set_index('ds')
TablePrec20 = TablePrec20.sort_index()
TablePrec20 = TablePrec20.rename(columns={'H20':'Prec'})
######### TableTemp21 ##############
TablePrec21 = pd.pivot_table(TablePrec, values=['H21'], index=["ds"], aggfunc={'H21':'first'})
TablePrec21 = TablePrec21.reset_index()
TablePrec21['ds'] = TablePrec21['ds'] + '-' + '210000'
TablePrec21['ds'] = pd.to_datetime(TablePrec21['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec21 = TablePrec21.set_index('ds')
TablePrec21 = TablePrec21.sort_index()
TablePrec21 = TablePrec21.rename(columns={'H21':'Prec'})
######### TableTemp22 ##############
TablePrec22 = pd.pivot_table(TablePrec, values=['H22'], index=["ds"], aggfunc={'H22':'first'})
TablePrec22 = TablePrec22.reset_index()
TablePrec22['ds'] = TablePrec22['ds'] + '-' + '220000'
TablePrec22['ds'] = pd.to_datetime(TablePrec22['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec22 = TablePrec22.set_index('ds')
TablePrec22 = TablePrec22.sort_index()
TablePrec22 = TablePrec22.rename(columns={'H22':'Prec'})
######### TableTemp23 ##############
TablePrec23 = pd.pivot_table(TablePrec, values=['H23'], index=["ds"], aggfunc={'H23':'first'})
TablePrec23 = TablePrec23.reset_index()
TablePrec23['ds'] = TablePrec23['ds'] + '-' + '230000'
TablePrec23['ds'] = pd.to_datetime(TablePrec23['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec23 = TablePrec23.set_index('ds')
TablePrec23 = TablePrec23.sort_index()
TablePrec23 = TablePrec23.rename(columns={'H23':'Prec'})
######### TableTemp0 ##############
TablePrec0 = pd.pivot_table(TablePrec, values=['H24'], index=["ds"], aggfunc={'H24':'first'})
TablePrec0 = TablePrec0.reset_index()
TablePrec0['ds'] = TablePrec0['ds'] + '-' + '000000'
TablePrec0['ds'] = pd.to_datetime(TablePrec0['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec0 = TablePrec0.set_index('ds')
TablePrec0 = TablePrec0.sort_index()
TablePrec0 = TablePrec0.rename(columns={'H24':'Prec'})

Data_PrecJul = pd.concat([TablePrec0,TablePrec1,TablePrec2,TablePrec3,TablePrec4,TablePrec5,TablePrec6,TablePrec7,TablePrec8,TablePrec9,TablePrec10,TablePrec11,TablePrec12,TablePrec13,TablePrec14,TablePrec15,TablePrec16,TablePrec17,TablePrec18,TablePrec19,TablePrec20,TablePrec21,TablePrec22,TablePrec23])
Data_PrecJul = Data_PrecJul.sort_index()
Data_PrecJul.tail(5)

,Prec
ds,
2020-07-31 19:00:00,0.0
2020-07-31 20:00:00,0.0
2020-07-31 21:00:00,0.0
2020-07-31 22:00:00,0.0
2020-07-31 23:00:00,0.0


# 4) Datos Metereologicos Agosto

In [21]:
DataMeteo = pd.read_csv("/home/jonathan/tesis/Data_Temperatura_Lluvia/ago_meteo20.csv", sep=';')
DataMeteo.head(1)

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H01,V01,...,H20,V20,H21,V21,H22,V22,H23,V23,H24,V24
0,28,79,102,81,28079102_81_98,2020,8,1,1.55,V,...,3.33,V,2.8,V,1.9,V,1.55,V,1.35,V


# 4.1) Procesamiento Datos de Temperatura
## Obtenemos datos de temperatura de la estación Chamberí

In [22]:
DataFiltrar = DataMeteo.loc[DataMeteo.loc[:,'ESTACION'] == 109]
DataTemp = DataFiltrar.loc[DataFiltrar.loc[:,'MAGNITUD'] == 83]
DataTemp = DataTemp.drop(columns=['PROVINCIA','MUNICIPIO','ESTACION','MAGNITUD','V01','V02','V03','V04','V05','V06','V07','V08','V09','V10','V11','V12','V13','V14','V15','V16','V17','V18','V19','V20','V21','V22','V23','V24'])
TableTemp = DataTemp
# Convertimos los elementos de las columnas "ANO" "MES" "DIA" a string
TableTemp['ANO']= TableTemp['ANO'].map(str)
TableTemp['MES']= TableTemp['MES'].map(str)
TableTemp['DIA']= TableTemp['DIA'].map(str)
# Ajustamos la columna ds con (-) para posteriormente obtener un timestamp
TableTemp['ds'] = TableTemp['ANO'] + '-' + TableTemp['MES'] + '-' + TableTemp['DIA']
TableTemp.tail(5)

,PUNTO_MUESTREO,ANO,MES,DIA,H01,H02,H03,H04,H05,H06,...,H16,H17,H18,H19,H20,H21,H22,H23,H24,ds
1173,28079109_83_98,2020,8,27,29.1,28.4,27.7,26.8,26.1,25.6,...,37.3,37.3,37.8,37.5,35.7,34.1,32.7,31.1,30.1,2020-8-27
1174,28079109_83_98,2020,8,28,29.3,28.3,27.7,27.1,26.8,25.6,...,32.8,33.1,33.7,32.5,30.3,27.7,26.0,24.6,23.4,2020-8-28
1175,28079109_83_98,2020,8,29,22.7,22.0,20.9,18.9,18.3,18.0,...,25.2,26.8,26.7,25.0,22.7,21.3,20.3,19.1,18.1,2020-8-29
1176,28079109_83_98,2020,8,30,17.3,16.5,16.0,15.7,15.0,14.5,...,23.9,23.1,24.2,24.0,23.2,21.2,20.0,19.3,18.6,2020-8-30
1177,28079109_83_98,2020,8,31,17.7,16.8,16.0,15.4,14.6,14.1,...,26.1,25.5,26.1,25.8,24.9,23.5,22.5,21.7,20.8,2020-8-31


# Obtenemos tablas de las 24 horas TableTemp#

In [23]:
######### TableTemp1 ##############
TableTemp1 = pd.pivot_table(TableTemp, values=['H01'], index=["ds"], aggfunc={'H01':'first'})
TableTemp1 = TableTemp1.reset_index()
TableTemp1['ds'] = TableTemp1['ds'] + '-' + '010000'
TableTemp1['ds'] = pd.to_datetime(TableTemp1['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp1 = TableTemp1.set_index('ds')
TableTemp1 = TableTemp1.sort_index()
TableTemp1 = TableTemp1.rename(columns={'H01':'Temp'})
######### TableTemp2 ##############
TableTemp2 = pd.pivot_table(TableTemp, values=['H02'], index=["ds"], aggfunc={'H02':'first'})
TableTemp2 = TableTemp2.reset_index()
TableTemp2['ds'] = TableTemp2['ds'] + '-' + '020000'
TableTemp2['ds'] = pd.to_datetime(TableTemp2['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp2 = TableTemp2.set_index('ds')
TableTemp2 = TableTemp2.sort_index()
TableTemp2 = TableTemp2.rename(columns={'H02':'Temp'})
######### TableTemp3 ##############
TableTemp3 = pd.pivot_table(TableTemp, values=['H03'], index=["ds"], aggfunc={'H03':'first'})
TableTemp3 = TableTemp3.reset_index()
TableTemp3['ds'] = TableTemp3['ds'] + '-' + '030000'
TableTemp3['ds'] = pd.to_datetime(TableTemp3['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp3 = TableTemp3.set_index('ds')
TableTemp3 = TableTemp3.sort_index()
TableTemp3 = TableTemp3.rename(columns={'H03':'Temp'})
######### TableTemp4 ##############
TableTemp4 = pd.pivot_table(TableTemp, values=['H04'], index=["ds"], aggfunc={'H04':'first'})
TableTemp4 = TableTemp4.reset_index()
TableTemp4['ds'] = TableTemp4['ds'] + '-' + '040000'
TableTemp4['ds'] = pd.to_datetime(TableTemp4['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp4 = TableTemp4.set_index('ds')
TableTemp4 = TableTemp4.sort_index()
TableTemp4 = TableTemp4.rename(columns={'H04':'Temp'})
######### TableTemp5 ##############
TableTemp5 = pd.pivot_table(TableTemp, values=['H05'], index=["ds"], aggfunc={'H05':'first'})
TableTemp5 = TableTemp5.reset_index()
TableTemp5['ds'] = TableTemp5['ds'] + '-' + '050000'
TableTemp5['ds'] = pd.to_datetime(TableTemp5['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp5 = TableTemp5.set_index('ds')
TableTemp5 = TableTemp5.sort_index()
TableTemp5 = TableTemp5.rename(columns={'H05':'Temp'})
######### TableTemp6 ##############
TableTemp6 = pd.pivot_table(TableTemp, values=['H06'], index=["ds"], aggfunc={'H06':'first'})
TableTemp6 = TableTemp6.reset_index()
TableTemp6['ds'] = TableTemp6['ds'] + '-' + '060000'
TableTemp6['ds'] = pd.to_datetime(TableTemp6['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp6 = TableTemp6.set_index('ds')
TableTemp6 = TableTemp6.sort_index()
TableTemp6 = TableTemp6.rename(columns={'H06':'Temp'})
######### TableTemp7 ##############
TableTemp7 = pd.pivot_table(TableTemp, values=['H07'], index=["ds"], aggfunc={'H07':'first'})
TableTemp7 = TableTemp7.reset_index()
TableTemp7['ds'] = TableTemp7['ds'] + '-' + '070000'
TableTemp7['ds'] = pd.to_datetime(TableTemp7['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp7 = TableTemp7.set_index('ds')
TableTemp7 = TableTemp7.sort_index()
TableTemp7 = TableTemp7.rename(columns={'H07':'Temp'})
######### TableTemp8 ##############
TableTemp8 = pd.pivot_table(TableTemp, values=['H08'], index=["ds"], aggfunc={'H08':'first'})
TableTemp8 = TableTemp8.reset_index()
TableTemp8['ds'] = TableTemp8['ds'] + '-' + '080000'
TableTemp8['ds'] = pd.to_datetime(TableTemp8['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp8 = TableTemp8.set_index('ds')
TableTemp8 = TableTemp8.sort_index()
TableTemp8 = TableTemp8.rename(columns={'H08':'Temp'})
######### TableTemp9 ##############
TableTemp9 = pd.pivot_table(TableTemp, values=['H09'], index=["ds"], aggfunc={'H09':'first'})
TableTemp9 = TableTemp9.reset_index()
TableTemp9['ds'] = TableTemp9['ds'] + '-' + '090000'
TableTemp9['ds'] = pd.to_datetime(TableTemp9['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp9 = TableTemp9.set_index('ds')
TableTemp9 = TableTemp9.sort_index()
TableTemp9 = TableTemp9.rename(columns={'H09':'Temp'})
######### TableTemp10 ##############
TableTemp10 = pd.pivot_table(TableTemp, values=['H10'], index=["ds"], aggfunc={'H10':'first'})
TableTemp10 = TableTemp10.reset_index()
TableTemp10['ds'] = TableTemp10['ds'] + '-' + '100000'
TableTemp10['ds'] = pd.to_datetime(TableTemp10['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp10 = TableTemp10.set_index('ds')
TableTemp10 = TableTemp10.sort_index()
TableTemp10 = TableTemp10.rename(columns={'H10':'Temp'})
######### TableTemp11 ##############
TableTemp11 = pd.pivot_table(TableTemp, values=['H11'], index=["ds"], aggfunc={'H11':'first'})
TableTemp11 = TableTemp11.reset_index()
TableTemp11['ds'] = TableTemp11['ds'] + '-' + '110000'
TableTemp11['ds'] = pd.to_datetime(TableTemp11['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp11 = TableTemp11.set_index('ds')
TableTemp11 = TableTemp11.sort_index()
TableTemp11 = TableTemp11.rename(columns={'H11':'Temp'})
######### TableTemp12 ##############
TableTemp12 = pd.pivot_table(TableTemp, values=['H12'], index=["ds"], aggfunc={'H12':'first'})
TableTemp12 = TableTemp12.reset_index()
TableTemp12['ds'] = TableTemp12['ds'] + '-' + '120000'
TableTemp12['ds'] = pd.to_datetime(TableTemp12['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp12 = TableTemp12.set_index('ds')
TableTemp12 = TableTemp12.sort_index()
TableTemp12 = TableTemp12.rename(columns={'H12':'Temp'})
######### TableTemp13 ##############
TableTemp13 = pd.pivot_table(TableTemp, values=['H13'], index=["ds"], aggfunc={'H13':'first'})
TableTemp13 = TableTemp13.reset_index()
TableTemp13['ds'] = TableTemp13['ds'] + '-' + '130000'
TableTemp13['ds'] = pd.to_datetime(TableTemp13['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp13 = TableTemp13.set_index('ds')
TableTemp13 = TableTemp13.sort_index()
TableTemp13 = TableTemp13.rename(columns={'H13':'Temp'})
######### TableTemp14 ##############
TableTemp14 = pd.pivot_table(TableTemp, values=['H14'], index=["ds"], aggfunc={'H14':'first'})
TableTemp14 = TableTemp14.reset_index()
TableTemp14['ds'] = TableTemp14['ds'] + '-' + '140000'
TableTemp14['ds'] = pd.to_datetime(TableTemp14['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp14 = TableTemp14.set_index('ds')
TableTemp14 = TableTemp14.sort_index()
TableTemp14 = TableTemp14.rename(columns={'H14':'Temp'})
######### TableTemp15 ##############
TableTemp15 = pd.pivot_table(TableTemp, values=['H15'], index=["ds"], aggfunc={'H15':'first'})
TableTemp15 = TableTemp15.reset_index()
TableTemp15['ds'] = TableTemp15['ds'] + '-' + '150000'
TableTemp15['ds'] = pd.to_datetime(TableTemp15['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp15 = TableTemp15.set_index('ds')
TableTemp15 = TableTemp15.sort_index()
TableTemp15 = TableTemp15.rename(columns={'H15':'Temp'})
######### TableTemp16 ##############
TableTemp16 = pd.pivot_table(TableTemp, values=['H16'], index=["ds"], aggfunc={'H16':'first'})
TableTemp16 = TableTemp16.reset_index()
TableTemp16['ds'] = TableTemp16['ds'] + '-' + '160000'
TableTemp16['ds'] = pd.to_datetime(TableTemp16['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp16 = TableTemp16.set_index('ds')
TableTemp16 = TableTemp16.sort_index()
TableTemp16 = TableTemp16.rename(columns={'H16':'Temp'})
######### TableTemp17 ##############
TableTemp17 = pd.pivot_table(TableTemp, values=['H17'], index=["ds"], aggfunc={'H17':'first'})
TableTemp17 = TableTemp17.reset_index()
TableTemp17['ds'] = TableTemp17['ds'] + '-' + '170000'
TableTemp17['ds'] = pd.to_datetime(TableTemp17['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp17 = TableTemp17.set_index('ds')
TableTemp17 = TableTemp17.sort_index()
TableTemp17 = TableTemp17.rename(columns={'H17':'Temp'})
######### TableTemp18 ##############
TableTemp18 = pd.pivot_table(TableTemp, values=['H18'], index=["ds"], aggfunc={'H18':'first'})
TableTemp18 = TableTemp18.reset_index()
TableTemp18['ds'] = TableTemp18['ds'] + '-' + '180000'
TableTemp18['ds'] = pd.to_datetime(TableTemp18['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp18 = TableTemp18.set_index('ds')
TableTemp18 = TableTemp18.sort_index()
TableTemp18 = TableTemp18.rename(columns={'H18':'Temp'})
######### TableTemp19 ##############
TableTemp19 = pd.pivot_table(TableTemp, values=['H19'], index=["ds"], aggfunc={'H19':'first'})
TableTemp19 = TableTemp19.reset_index()
TableTemp19['ds'] = TableTemp19['ds'] + '-' + '190000'
TableTemp19['ds'] = pd.to_datetime(TableTemp19['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp19 = TableTemp19.set_index('ds')
TableTemp19 = TableTemp19.sort_index()
TableTemp19 = TableTemp19.rename(columns={'H19':'Temp'})
######### TableTemp20 ##############
TableTemp20 = pd.pivot_table(TableTemp, values=['H20'], index=["ds"], aggfunc={'H20':'first'})
TableTemp20 = TableTemp20.reset_index()
TableTemp20['ds'] = TableTemp20['ds'] + '-' + '200000'
TableTemp20['ds'] = pd.to_datetime(TableTemp20['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp20 = TableTemp20.set_index('ds')
TableTemp20 = TableTemp20.sort_index()
TableTemp20 = TableTemp20.rename(columns={'H20':'Temp'})
######### TableTemp21 ##############
TableTemp21 = pd.pivot_table(TableTemp, values=['H21'], index=["ds"], aggfunc={'H21':'first'})
TableTemp21 = TableTemp21.reset_index()
TableTemp21['ds'] = TableTemp21['ds'] + '-' + '210000'
TableTemp21['ds'] = pd.to_datetime(TableTemp21['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp21 = TableTemp21.set_index('ds')
TableTemp21 = TableTemp21.sort_index()
TableTemp21 = TableTemp21.rename(columns={'H21':'Temp'})
######### TableTemp22 ##############
TableTemp22 = pd.pivot_table(TableTemp, values=['H22'], index=["ds"], aggfunc={'H22':'first'})
TableTemp22 = TableTemp22.reset_index()
TableTemp22['ds'] = TableTemp22['ds'] + '-' + '220000'
TableTemp22['ds'] = pd.to_datetime(TableTemp22['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp22 = TableTemp22.set_index('ds')
TableTemp22 = TableTemp22.sort_index()
TableTemp22 = TableTemp22.rename(columns={'H22':'Temp'})
######### TableTemp23 ##############
TableTemp23 = pd.pivot_table(TableTemp, values=['H23'], index=["ds"], aggfunc={'H23':'first'})
TableTemp23 = TableTemp23.reset_index()
TableTemp23['ds'] = TableTemp23['ds'] + '-' + '230000'
TableTemp23['ds'] = pd.to_datetime(TableTemp23['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp23 = TableTemp23.set_index('ds')
TableTemp23 = TableTemp23.sort_index()
TableTemp23 = TableTemp23.rename(columns={'H23':'Temp'})
######### TableTemp0 ##############
TableTemp0 = pd.pivot_table(TableTemp, values=['H24'], index=["ds"], aggfunc={'H24':'first'})
TableTemp0 = TableTemp0.reset_index()
TableTemp0['ds'] = TableTemp0['ds'] + '-' + '000000'
TableTemp0['ds'] = pd.to_datetime(TableTemp0['ds'], format='%Y-%m-%d-%H%M%S')
TableTemp0 = TableTemp0.set_index('ds')
TableTemp0 = TableTemp0.sort_index()
TableTemp0 = TableTemp0.rename(columns={'H24':'Temp'})

# Concatenamos todas las tablas en una sola

In [24]:
Data_TempAgo = pd.concat([TableTemp0,TableTemp1,TableTemp2,TableTemp3,TableTemp4,TableTemp5,TableTemp6,TableTemp7,TableTemp8,TableTemp9,TableTemp10,TableTemp11,TableTemp12,TableTemp13,TableTemp14,TableTemp15,TableTemp16,TableTemp17,TableTemp18,TableTemp19,TableTemp20,TableTemp21,TableTemp22,TableTemp23])
Data_TempAgo = Data_TempAgo.sort_index()
Data_TempAgo.tail(5)

,Temp
ds,
2020-08-31 19:00:00,25.8
2020-08-31 20:00:00,24.9
2020-08-31 21:00:00,23.5
2020-08-31 22:00:00,22.5
2020-08-31 23:00:00,21.7


# 4.2) Procesamiento Datos Precipitación
## Obtenemos datos de Precipitación de la estación Hortaleza 

In [25]:
DataFiltrar2 = DataMeteo.loc[DataMeteo.loc[:,'ESTACION'] == 107]
DataPrec = DataFiltrar2.loc[DataFiltrar2.loc[:,'MAGNITUD'] == 89]
DataPrec = DataPrec.drop(columns=['PROVINCIA','MUNICIPIO','ESTACION','MAGNITUD','V01','V02','V03','V04','V05','V06','V07','V08','V09','V10','V11','V12','V13','V14','V15','V16','V17','V18','V19','V20','V21','V22','V23','V24'])
TablePrec = DataPrec
# Convertimos los elementos de las columnas "ANO" "MES" "DIA" a string
TablePrec['ANO']= TablePrec['ANO'].map(str)
TablePrec['MES']= TablePrec['MES'].map(str)
TablePrec['DIA']= TablePrec['DIA'].map(str)
# Ajustamos la columna ds con (-) para posteriormente obtener un timestamp
TablePrec['ds'] = TablePrec['ANO'] + '-' + TablePrec['MES'] + '-' + TablePrec['DIA']
TablePrec.tail(5)

,PUNTO_MUESTREO,ANO,MES,DIA,H01,H02,H03,H04,H05,H06,...,H16,H17,H18,H19,H20,H21,H22,H23,H24,ds
925,28079107_89_98,2020,8,27,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-8-27
926,28079107_89_98,2020,8,28,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-8-28
927,28079107_89_98,2020,8,29,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,2020-8-29
928,28079107_89_98,2020,8,30,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-8-30
929,28079107_89_98,2020,8,31,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-8-31


In [26]:
######### TableTemp1 ##############
TablePrec1 = pd.pivot_table(TablePrec, values=['H01'], index=["ds"], aggfunc={'H01':'first'})
TablePrec1 = TablePrec1.reset_index()
TablePrec1['ds'] = TablePrec1['ds'] + '-' + '010000'
TablePrec1['ds'] = pd.to_datetime(TablePrec1['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec1 = TablePrec1.set_index('ds')
TablePrec1 = TablePrec1.sort_index()
TablePrec1 = TablePrec1.rename(columns={'H01':'Prec'})
######### TableTemp2 ##############
TablePrec2 = pd.pivot_table(TablePrec, values=['H02'], index=["ds"], aggfunc={'H02':'first'})
TablePrec2 = TablePrec2.reset_index()
TablePrec2['ds'] = TablePrec2['ds'] + '-' + '020000'
TablePrec2['ds'] = pd.to_datetime(TablePrec2['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec2 = TablePrec2.set_index('ds')
TablePrec2 = TablePrec2.sort_index()
TablePrec2 = TablePrec2.rename(columns={'H02':'Prec'})
######### TableTemp3 ##############
TablePrec3 = pd.pivot_table(TablePrec, values=['H03'], index=["ds"], aggfunc={'H03':'first'})
TablePrec3 = TablePrec3.reset_index()
TablePrec3['ds'] = TablePrec3['ds'] + '-' + '030000'
TablePrec3['ds'] = pd.to_datetime(TablePrec3['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec3 = TablePrec3.set_index('ds')
TablePrec3 = TablePrec3.sort_index()
TablePrec3 = TablePrec3.rename(columns={'H03':'Prec'})
######### TableTemp4 ##############
TablePrec4 = pd.pivot_table(TablePrec, values=['H04'], index=["ds"], aggfunc={'H04':'first'})
TablePrec4 = TablePrec4.reset_index()
TablePrec4['ds'] = TablePrec4['ds'] + '-' + '040000'
TablePrec4['ds'] = pd.to_datetime(TablePrec4['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec4 = TablePrec4.set_index('ds')
TablePrec4 = TablePrec4.sort_index()
TablePrec4 = TablePrec4.rename(columns={'H04':'Prec'})
######### TableTemp5 ##############
TablePrec5 = pd.pivot_table(TablePrec, values=['H05'], index=["ds"], aggfunc={'H05':'first'})
TablePrec5 = TablePrec5.reset_index()
TablePrec5['ds'] = TablePrec5['ds'] + '-' + '050000'
TablePrec5['ds'] = pd.to_datetime(TablePrec5['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec5 = TablePrec5.set_index('ds')
TablePrec5 = TablePrec5.sort_index()
TablePrec5 = TablePrec5.rename(columns={'H05':'Prec'})
######### TableTemp6 ##############
TablePrec6 = pd.pivot_table(TablePrec, values=['H06'], index=["ds"], aggfunc={'H06':'first'})
TablePrec6 = TablePrec6.reset_index()
TablePrec6['ds'] = TablePrec6['ds'] + '-' + '060000'
TablePrec6['ds'] = pd.to_datetime(TablePrec6['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec6 = TablePrec6.set_index('ds')
TablePrec6 = TablePrec6.sort_index()
TablePrec6 = TablePrec6.rename(columns={'H06':'Prec'})
######### TableTemp7 ##############
TablePrec7 = pd.pivot_table(TablePrec, values=['H07'], index=["ds"], aggfunc={'H07':'first'})
TablePrec7 = TablePrec7.reset_index()
TablePrec7['ds'] = TablePrec7['ds'] + '-' + '070000'
TablePrec7['ds'] = pd.to_datetime(TablePrec7['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec7 = TablePrec7.set_index('ds')
TablePrec7 = TablePrec7.sort_index()
TablePrec7 = TablePrec7.rename(columns={'H07':'Prec'})
######### TableTemp8 ##############
TablePrec8 = pd.pivot_table(TablePrec, values=['H08'], index=["ds"], aggfunc={'H08':'first'})
TablePrec8 = TablePrec8.reset_index()
TablePrec8['ds'] = TablePrec8['ds'] + '-' + '080000'
TablePrec8['ds'] = pd.to_datetime(TablePrec8['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec8 = TablePrec8.set_index('ds')
TablePrec8 = TablePrec8.sort_index()
TablePrec8 = TablePrec8.rename(columns={'H08':'Prec'})
######### TableTemp9 ##############
TablePrec9 = pd.pivot_table(TablePrec, values=['H09'], index=["ds"], aggfunc={'H09':'first'})
TablePrec9 = TablePrec9.reset_index()
TablePrec9['ds'] = TablePrec9['ds'] + '-' + '090000'
TablePrec9['ds'] = pd.to_datetime(TablePrec9['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec9 = TablePrec9.set_index('ds')
TablePrec9 = TablePrec9.sort_index()
TablePrec9 = TablePrec9.rename(columns={'H09':'Prec'})
######### TableTemp10 ##############
TablePrec10 = pd.pivot_table(TablePrec, values=['H10'], index=["ds"], aggfunc={'H10':'first'})
TablePrec10 = TablePrec10.reset_index()
TablePrec10['ds'] = TablePrec10['ds'] + '-' + '100000'
TablePrec10['ds'] = pd.to_datetime(TablePrec10['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec10 = TablePrec10.set_index('ds')
TablePrec10 = TablePrec10.sort_index()
TablePrec10 = TablePrec10.rename(columns={'H10':'Prec'})
######### TableTemp11 ##############
TablePrec11 = pd.pivot_table(TablePrec, values=['H11'], index=["ds"], aggfunc={'H11':'first'})
TablePrec11 = TablePrec11.reset_index()
TablePrec11['ds'] = TablePrec11['ds'] + '-' + '110000'
TablePrec11['ds'] = pd.to_datetime(TablePrec11['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec11 = TablePrec11.set_index('ds')
TablePrec11 = TablePrec11.sort_index()
TablePrec11 = TablePrec11.rename(columns={'H11':'Prec'})
######### TableTemp12 ##############
TablePrec12 = pd.pivot_table(TablePrec, values=['H12'], index=["ds"], aggfunc={'H12':'first'})
TablePrec12 = TablePrec12.reset_index()
TablePrec12['ds'] = TablePrec12['ds'] + '-' + '120000'
TablePrec12['ds'] = pd.to_datetime(TablePrec12['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec12 = TablePrec12.set_index('ds')
TablePrec12 = TablePrec12.sort_index()
TablePrec12 = TablePrec12.rename(columns={'H12':'Prec'})
######### TableTemp13 ##############
TablePrec13 = pd.pivot_table(TablePrec, values=['H13'], index=["ds"], aggfunc={'H13':'first'})
TablePrec13 = TablePrec13.reset_index()
TablePrec13['ds'] = TablePrec13['ds'] + '-' + '130000'
TablePrec13['ds'] = pd.to_datetime(TablePrec13['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec13 = TablePrec13.set_index('ds')
TablePrec13 = TablePrec13.sort_index()
TablePrec13 = TablePrec13.rename(columns={'H13':'Prec'})
######### TableTemp14 ##############
TablePrec14 = pd.pivot_table(TablePrec, values=['H14'], index=["ds"], aggfunc={'H14':'first'})
TablePrec14 = TablePrec14.reset_index()
TablePrec14['ds'] = TablePrec14['ds'] + '-' + '140000'
TablePrec14['ds'] = pd.to_datetime(TablePrec14['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec14 = TablePrec14.set_index('ds')
TablePrec14 = TablePrec14.sort_index()
TablePrec14 = TablePrec14.rename(columns={'H14':'Prec'})
######### TableTemp15 ##############
TablePrec15 = pd.pivot_table(TablePrec, values=['H15'], index=["ds"], aggfunc={'H15':'first'})
TablePrec15 = TablePrec15.reset_index()
TablePrec15['ds'] = TablePrec15['ds'] + '-' + '150000'
TablePrec15['ds'] = pd.to_datetime(TablePrec15['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec15 = TablePrec15.set_index('ds')
TablePrec15 = TablePrec15.sort_index()
TablePrec15 = TablePrec15.rename(columns={'H15':'Prec'})
######### TableTemp16 ##############
TablePrec16 = pd.pivot_table(TablePrec, values=['H16'], index=["ds"], aggfunc={'H16':'first'})
TablePrec16 = TablePrec16.reset_index()
TablePrec16['ds'] = TablePrec16['ds'] + '-' + '160000'
TablePrec16['ds'] = pd.to_datetime(TablePrec16['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec16 = TablePrec16.set_index('ds')
TablePrec16 = TablePrec16.sort_index()
TablePrec16 = TablePrec16.rename(columns={'H16':'Prec'})
######### TableTemp17 ##############
TablePrec17 = pd.pivot_table(TablePrec, values=['H17'], index=["ds"], aggfunc={'H17':'first'})
TablePrec17 = TablePrec17.reset_index()
TablePrec17['ds'] = TablePrec17['ds'] + '-' + '170000'
TablePrec17['ds'] = pd.to_datetime(TablePrec17['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec17 = TablePrec17.set_index('ds')
TablePrec17 = TablePrec17.sort_index()
TablePrec17 = TablePrec17.rename(columns={'H17':'Prec'})
######### TableTemp18 ##############
TablePrec18 = pd.pivot_table(TablePrec, values=['H18'], index=["ds"], aggfunc={'H18':'first'})
TablePrec18 = TablePrec18.reset_index()
TablePrec18['ds'] = TablePrec18['ds'] + '-' + '180000'
TablePrec18['ds'] = pd.to_datetime(TablePrec18['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec18 = TablePrec18.set_index('ds')
TablePrec18 = TablePrec18.sort_index()
TablePrec18 = TablePrec18.rename(columns={'H18':'Prec'})
######### TableTemp19 ##############
TablePrec19 = pd.pivot_table(TablePrec, values=['H19'], index=["ds"], aggfunc={'H19':'first'})
TablePrec19 = TablePrec19.reset_index()
TablePrec19['ds'] = TablePrec19['ds'] + '-' + '190000'
TablePrec19['ds'] = pd.to_datetime(TablePrec19['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec19 = TablePrec19.set_index('ds')
TablePrec19 = TablePrec19.sort_index()
TablePrec19 = TablePrec19.rename(columns={'H19':'Prec'})
######### TableTemp20 ##############
TablePrec20 = pd.pivot_table(TablePrec, values=['H20'], index=["ds"], aggfunc={'H20':'first'})
TablePrec20 = TablePrec20.reset_index()
TablePrec20['ds'] = TablePrec20['ds'] + '-' + '200000'
TablePrec20['ds'] = pd.to_datetime(TablePrec20['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec20 = TablePrec20.set_index('ds')
TablePrec20 = TablePrec20.sort_index()
TablePrec20 = TablePrec20.rename(columns={'H20':'Prec'})
######### TableTemp21 ##############
TablePrec21 = pd.pivot_table(TablePrec, values=['H21'], index=["ds"], aggfunc={'H21':'first'})
TablePrec21 = TablePrec21.reset_index()
TablePrec21['ds'] = TablePrec21['ds'] + '-' + '210000'
TablePrec21['ds'] = pd.to_datetime(TablePrec21['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec21 = TablePrec21.set_index('ds')
TablePrec21 = TablePrec21.sort_index()
TablePrec21 = TablePrec21.rename(columns={'H21':'Prec'})
######### TableTemp22 ##############
TablePrec22 = pd.pivot_table(TablePrec, values=['H22'], index=["ds"], aggfunc={'H22':'first'})
TablePrec22 = TablePrec22.reset_index()
TablePrec22['ds'] = TablePrec22['ds'] + '-' + '220000'
TablePrec22['ds'] = pd.to_datetime(TablePrec22['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec22 = TablePrec22.set_index('ds')
TablePrec22 = TablePrec22.sort_index()
TablePrec22 = TablePrec22.rename(columns={'H22':'Prec'})
######### TableTemp23 ##############
TablePrec23 = pd.pivot_table(TablePrec, values=['H23'], index=["ds"], aggfunc={'H23':'first'})
TablePrec23 = TablePrec23.reset_index()
TablePrec23['ds'] = TablePrec23['ds'] + '-' + '230000'
TablePrec23['ds'] = pd.to_datetime(TablePrec23['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec23 = TablePrec23.set_index('ds')
TablePrec23 = TablePrec23.sort_index()
TablePrec23 = TablePrec23.rename(columns={'H23':'Prec'})
######### TableTemp0 ##############
TablePrec0 = pd.pivot_table(TablePrec, values=['H24'], index=["ds"], aggfunc={'H24':'first'})
TablePrec0 = TablePrec0.reset_index()
TablePrec0['ds'] = TablePrec0['ds'] + '-' + '000000'
TablePrec0['ds'] = pd.to_datetime(TablePrec0['ds'], format='%Y-%m-%d-%H%M%S')
TablePrec0 = TablePrec0.set_index('ds')
TablePrec0 = TablePrec0.sort_index()
TablePrec0 = TablePrec0.rename(columns={'H24':'Prec'})

Data_PrecAgo = pd.concat([TablePrec0,TablePrec1,TablePrec2,TablePrec3,TablePrec4,TablePrec5,TablePrec6,TablePrec7,TablePrec8,TablePrec9,TablePrec10,TablePrec11,TablePrec12,TablePrec13,TablePrec14,TablePrec15,TablePrec16,TablePrec17,TablePrec18,TablePrec19,TablePrec20,TablePrec21,TablePrec22,TablePrec23])
Data_PrecAgo = Data_PrecAgo.sort_index()
Data_PrecAgo.tail(5)

,Prec
ds,
2020-08-31 19:00:00,0.0
2020-08-31 20:00:00,0.0
2020-08-31 21:00:00,0.0
2020-08-31 22:00:00,0.0
2020-08-31 23:00:00,0.0


# Obtenemos el Dataframe Final de temperatura y precipitacion

In [27]:
Data_Temp = pd.concat([Data_TempMay,Data_TempJun,Data_TempJul,Data_TempAgo])
Data_Temp = Data_Temp.sort_index()
Data_Temp.tail(5)

,Temp
ds,
2020-08-31 19:00:00,25.8
2020-08-31 20:00:00,24.9
2020-08-31 21:00:00,23.5
2020-08-31 22:00:00,22.5
2020-08-31 23:00:00,21.7


In [28]:
Data_Prec = pd.concat([Data_PrecMay,Data_PrecJun,Data_PrecJul,Data_PrecAgo])
Data_Prec = Data_Prec.sort_index()
Data_Prec.tail(5)

,Prec
ds,
2020-08-31 19:00:00,0.0
2020-08-31 20:00:00,0.0
2020-08-31 21:00:00,0.0
2020-08-31 22:00:00,0.0
2020-08-31 23:00:00,0.0


In [40]:
# Data_Prec.iloc[2900:2950]

In [29]:
# Data_Temp.to_csv("DataTempMayAgo.csv")
Data_Prec.to_csv("DataPrecMayAgo.csv")